<a href="https://colab.research.google.com/github/gherbin/ComputerVisionKUL/blob/master/CV_Group9_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hi there! 
Welcome to this Colab where we'll dig into some Computer Vision fancy stuff!

The goals of this notebook is to perform faces classification and identification. To reach that goals we will first:
- retrieve training and test images
- build two features
    1. handcrafted feature: Histogram of Oriented Gradients
    2. Learnt from the data: Principal Component Analysis

Then, we will train a model based on each feature and compare the classification and identification results.

---

Several libraries will be extensively used as optimized for some techniques we need. However, at first, some of the key functionalities will be coded as to provide a better view of what really happens being the calls to library functions.



## Import all required packages

In [0]:
import os
import cv2
import tarfile
import zipfile
import shutil


import numpy as np
import random
import logging

from urllib import request
from socket import timeout
from urllib.error import HTTPError, URLError

from google.colab import drive
from google.colab.patches import cv2_imshow

from distutils.dir_util import copy_tree

from skimage.feature import hog as skimage_feature_hog
from sklearn.decomposition import PCA as sklearn_decomposition_PCA
from skimage import exposure
from sklearn.metrics import mean_squared_error

from math import sqrt

from matplotlib import pyplot as plt
from matplotlib.gridspec import GridSpec
from matplotlib.offsetbox import OffsetImage, AnnotationBbox
%matplotlib inline

from scipy.interpolate import RectBivariateSpline
from scipy.linalg import svd as scipy_linalg_svd

logging.basicConfig(level=logging.INFO)

mpl_logger = logging.getLogger("matplotlib")
mpl_logger.setLevel(logging.WARNING)


## Parameters

As all computerized systems, several parameters help in defining how the system should react. 
Those parameters are centralized here.

In [0]:
base_path = "/content/sample_data/CV__Group_assignment"
path_datasets = r"/content/datasets/"
path_discard = r"/content/discard/"
path_database = r"/content/DATABASE/"

need_vgg_download = False
confirmation_db_renewal = False
to_drive_confirmation = False
to_drive_confirmation_vgg = False

load_from_local_drive = True # allows downloading the source images directly from the archive in github repository (see "important note")

show = True # similar as global verbose parameter, for images (when custom functions allows it)

sq_size = 64 # square size used -> shall be smaller than the output of get_min_size(faces_cropped) # assert sq_size <= min(get_min_size(faces_cropped))

color = False

if color:
    my_color_map = plt.cm.viridis
else:
    my_color_map = plt.cm.gray



## Utils functions



In [0]:
def pretty_return_dict_size(my_dict):
    """
    returns a string containing the different size of the elements of a dict
    """
    output_list = ["\n"]
    for k in my_dict.keys():
        output_list.append(str(k))
        output_list.append(":")
        output_list.append(str(len(my_dict[k])))
        output_list.append("\n")
    return ''.join(output_list)

def show_images_from_dict(my_dict, show_index = False):
    """
    shows the images contained in a dictionary, going through all keys
    """
    for k in my_dict.keys():
        logging.debug("@------------------- Images of " + str(k) + " -------------------@")
        index = 0
        for img in my_dict[k]:
            if show_index:
                logging.debug("Image index: " + str(index))
                index+=1
            cv2_imshow(img)
            logging.debug("Shape = " + str(img.shape))
def get_min_size(images_dict):
    """
    returns the minimum size of images contained in the images_dict input
    """
    min_rows, min_cols = float("inf"), float("inf")
    max_rows, max_cols = 0, 0
    for person in persons:
        for src in images_dict[person]:
            r, c = src.shape[0], src.shape[1]    
            min_rows = min(min_rows, r)
            max_rows = max(max_rows, r)
            min_cols = min(min_cols, c)
            max_cols = max(max_cols, c)
    logging.info("smallest px numbers (row, cols) = " + str((min_rows,min_cols)))
    return min_rows, min_cols
        
def my_reshape(image_vector, sq_size, color):
    """
    returns a reshape version of an image represented as an image array, depending of the color parameter.
    If color is True, it returns a colored RGB format image of size (sq_size x sq_size) (useable as is by matplotlib)
    If color is False, it returns a grayscale image (sq_size x sq_size)
    """
    flattened = image_vector.ndim == 1
    if flattened:
        if color:
            img_reshaped = (np.reshape(image_vector, (sq_size, sq_size, 3))).astype('uint8')
            return cv2.cvtColor(img_reshaped, cv2.COLOR_BGR2RGB)
        else:
            return np.reshape(image_vector, (sq_size, sq_size))
    else:
        if color:
            img_reshaped = (np.reshape(image_vector, (sq_size, sq_size, 3))).astype('uint8')
            return cv2.cvtColor(img_reshaped, cv2.COLOR_BGR2RGB)
        else:
            return image_vector

def get_matrix_from_set(images_set, color, sq_size = 64, flatten = True):
    """
    from images_set (training_set or test_set), create and fill in matrix so that it contains the input data.
    if flatten, then the matrix contains images represented in 1D
    """

    # init output
    matrix = None
    nb_faces = sum([len(images_set[x]) for x in images_set if isinstance(images_set[x], list)])
    # depending on mode, select appropriate size items. N
    
    if color and flatten:
        matrix = np.empty((nb_faces, sq_size*sq_size*3)) # *3 => color images
    elif color and (not flatten):
        matrix = np.empty((nb_faces, sq_size, sq_size, 3))
    elif (not color) and flatten:
        matrix = np.empty((nb_faces, sq_size*sq_size))
    elif (not color) and (not flatten):
        matrix = np.empty((nb_faces, sq_size,sq_size ))
    else:
        raise RuntimeError

    i = 0
    for person in persons:
        for src in images_set[person]:
            src_rescaled = cv2.resize(src, (sq_size,sq_size))
            if color and flatten:
                matrix[i,:] = src_rescaled.flatten()
            elif color and (not flatten):
                matrix[i,:,:,:] = src_rescaled
            elif (not color) and flatten:
                matrix[i,:] = cv2.cvtColor(src_rescaled, cv2.COLOR_BGR2GRAY).flatten()
            elif (not color) and (not flatten):
                matrix[i,:,:] = cv2.cvtColor(src_rescaled, cv2.COLOR_BGR2GRAY)

            i +=1
    return matrix

def plot_matrix(images_matrix, color, my_color_map, h=8, w=5, transpose = False, return_figure = False):
    # if figure is not None:
    #     fig = figure
    #     fig.set_size_inches(w, h)
    # else:
    fig = plt.figure(figsize=(w,h)) 
    fig.subplots_adjust(left=0, right=1, bottom=0, top=1, hspace=0.05, wspace=0.05) 
    # plot the faces, each image is 64 by 64 pixels 

    if transpose:
        images_matrix_used = images_matrix.T.copy()
    else:
        images_matrix_used = images_matrix.copy()

    i=0
    for img_vector in images_matrix_used: 
        ax = fig.add_subplot(h, w, i+1, xticks=[], yticks=[]) 
        ax.imshow(my_reshape(img_vector, sq_size, color), cmap = my_color_map, interpolation='nearest') 
        i+=1
    plt.show()

    if return_figure:
        return fig

## Inputs

* The very first input of the system is an archive containing several text file containing each 1000 weblinks to images. This archive is downloaded from [here](http://www.robots.ox.ac.uk/~vgg/data/vgg_face) and extracted locally in `/content/sample_data/CV__Group_assignment` folder. We download and extract it only if needed

* To extract faces in the images, we download the Haarcascade model




In [0]:
if not os.path.isdir(base_path):
  os.makedirs(base_path)

if need_vgg_download:
    vgg_face_dataset_url = "http://www.robots.ox.ac.uk/~vgg/data/vgg_face/vgg_face_dataset.tar.gz"
    with request.urlopen(vgg_face_dataset_url) as r, open(os.path.join(base_path, "vgg_face_dataset.tar.gz"), 'wb') as f:
        f.write(r.read())

    with tarfile.open(os.path.join(base_path, "vgg_face_dataset.tar.gz")) as f:
        f.extractall(os.path.join(base_path))


trained_haarcascade_url = "https://raw.githubusercontent.com/opencv/opencv/master/data/haarcascades/haarcascade_frontalface_default.xml"
with request.urlopen(trained_haarcascade_url) as r, open(os.path.join(base_path, "haarcascade_frontalface_default.xml"), 'wb') as f:
    f.write(r.read())
logging.info("Downloaded haarcascade_frontalface_default")

# Data Retrieval

This tutorial will extensively use images from four different actors. The images are selected (pseudo-)randomly.

The movie stars are (chosen quite randomly as well):
1.   personA: Emma Stone
2.   personB: Bradley Cooper
3.   personC: Jane Levy
4.   personD: Marc Blucas


Process to get datasets images:
1. Randomly pick N images (60 for persons A and B, and 30 for persons C and D) images from the list of 1000 images provided in the textfile. 
3. Reject some i images that are not appropriated (see rejection step lated) for each person. i may of course be different for all actors.
2. Select M images randomly out of the (N-i) images obtained for each persons:
    - M=30 for personA and personB (Training and Test),
    - M=10 for personC and personD (Test only)


---


**[IMPORTANT NOTE]**

If nothing else is set up, getting the N images require to perform a url request on websites we do not control. This is risky, as for any reason, the target website could be modified, not responding, responding too slowly, have removed the picture of interest, ...
To prevent such issue, this tutorial provide the code to do things differently.
- the first time (with some parameters below properly set), the source images are downloaded from the website (retrieving errors, skipping too slow website, etc.)
- the images, downloaded, are then saved and zipped with a logfile
- this zip archive is then uploaded on my personal Github account, as a public file

It leads to a controlled database constaining the source images, and ensure reproducibility during the different test run.

*Three remarks*
1. only the original files are stored in the archive in the ZIP. Those files were selected randomly, using a random number generator.
2. the curation of the source files, the face cropping, and selection between training and test sets is still done at every run of this notebook.
3. the code dedicated to the archiving and saving part will not be detailed (while yet provided) in this notebook, but surely, you are welcome to contact me for more details using geoffroy.herbin@student.kuleuven.be.

---



Start from clean sheet


In [0]:
try:
    shutil.rmtree(path_database)
    shutil.rmtree(path_datasets)
    shutil.rmtree(path_discard)
except:
    pass

Create required folders

In [0]:
file_info = path_database+ r"info_retrieved.txt"

try: 
    os.mkdir(path_database)
    os.mkdir(path_datasets) 
    os.mkdir(path_discard)
except OSError as error: 
    logging.error(error) 



Instead of randomly download from web, download images from a "clean" and controlled repository (in [github](https://raw.githubusercontent.com/gherbin/cv_group9_database_replica/master/DATABASE-20200318T142918Z-001.zip) ), dedicated for this notebook. It ensures reproducibility and accessibility to the 180 input images.


In [0]:
if load_from_local_drive:
    
    !wget https://raw.githubusercontent.com/gherbin/cv_group9_database_replica/master/DATABASE-20200318T142918Z-001.zip

    with zipfile.ZipFile("DATABASE-20200318T142918Z-001.zip", 'r') as zip_ref:
        zip_ref.extractall()
    !rm -r "DATABASE-20200318T142918Z-001.zip"

path_, dirs_, files = next(os.walk(path_database))
if len(files) == 180+1:
    logging.info("Successful database retrieval")
elif load_from_local_drive:
    logging.error("Most Likely problem with database retrieval, number of files = " + str(len(files)))
else:
    logging.info("No database images retrieved yet")

###Definition of several data structures

`images_size` : dictionary containing the number of images to first get from the web

`persons` : list containing the names of the four persons (the names actually are the name of the text file in original database)

`images`: dictionary containing the source images. The keys of the dictionary are the names of the four persons of interest

In [0]:
personA = "Emma_Stone.txt"
personC = "Jane_Levy.txt"
personB = "Bradley_Cooper.txt"
personD = "Marc_Blucas.txt"
persons = [personA, personB, personC, personD]
datasets_dict = {}
images_size = {}
images_size[personA] = 60
images_size[personB] = 60
images_size[personC] = 30
images_size[personD] = 30

total_images_size = sum(images_size.values())

# Dictionary containing the ids of the pictures downloaded from internet
vgg_ids = {}
for p in persons:
    vgg_ids[p] = []


If `confirmation_db_renewal` is `True`, the following code picks randomly (based on a seed being the name of the person) the images from the web.

For a normal run, if the user does not want to change the original sourced data, `confirmation_db_renewal` should remain `False` (aka *change at your own risk* ;-) )



In [0]:
if confirmation_db_renewal:
    try:
        shutil.rmtree(path_database)
    except:
        pass 
    try:
        os.mkdir(path_database)
    except:
        pass 

    fo = open(file_info, "w+")

    # images = {}
    # images_nominal_indices = {}
    for person in persons:
        logging.debug("Taking care of: " + str(person))
        random.seed(person)
        # print(hash(person))
        images_ = []
        # images_nominal_indices_ = []
        prev_index = []


        with open(os.path.join(base_path, "vgg_face_dataset", "files", person), 'r') as f:
            lines = f.readlines()       
        

        while len(images_) < images_size[person]:
            index = random.randrange(0, 1000)
            logging.debug("Index = " + str(index))
            if index in prev_index:
                logging.debug("Index = " + str(index) + " => already there")
                continue
            else:
                prev_index.append(index)
                line = lines[index]
                # only curated data
                if int(line.split(" ")[8]) == 1:
                    url = line[line.find("http://"): line.find(".jpg") + 4]
                    logging.debug("URL > \"" + str(url))
                    try:
                        res = request.urlopen(url, timeout = 1)
                        img = np.asarray(bytearray(res.read()), dtype="uint8")
                        img = cv2.imdecode(img, cv2.IMREAD_COLOR)

                        h, w = img.shape[:2]
                        cv2_imshow(cv2.resize(img, (w//4, h//4)))
                        # images_nominal_indices_.append(index)

                        filename = path_database +  str(index) + "_" + str(person.split(".")[0]) + ".jpg"

                        value = cv2.imwrite(filename, img) 
                        # logging.debug("saved in DB: " + str(filename))
                        images_.append(img)
                        fo.write(line)
                    except ValueError as e:
                            logging.error("Value Error >" + str(e))
                    except (HTTPError, URLError) as e:
                            logging.error('ERROR RETRIEVING URL >' + str(e))
                    except timeout:
                            logging.error('socket timed out - URL %s', str(url))
                    except cv2.error as e: 
                            logging.error("ERROR WRITING FILE IN DB  >" + str(e))
                    except:
                        logging.error("Weird exception : " + str(line))
                else:
                    logging.debug("File not curated => rejected (id = " + str(index) + " )")    
                
                # images[person] = images_
                # images_nominal_indices[person] = images_nominal_indices_

    fo.close()
else:
    logging.warning("If you really want to erase and renew the database, please change first the \"confirmation\" boolean variable, at the beginning of this cell")


From the logfile in the archive, extract the information and fill the dictionary containing all the images `images`.



In [0]:
with open(file_info, 'r') as f: 
    lines = f.readlines()

assert len(lines)==total_images_size, "amount of lines in file incompatible" 

images = {}

for p in persons:
    images[p] = []


images_index = {}
for running_index in range(len(lines)):
    if running_index in range(0,images_size[personA]):
        p = personA
    elif running_index in range(images_size[personA],images_size[personA]+images_size[personB]):
        p = personB
    elif running_index in range(images_size[personA]+images_size[personB],images_size[personA]+images_size[personB]+images_size[personC]):
        p = personC
    elif running_index in range(images_size[personA]+images_size[personB]+images_size[personC],total_images_size):
        p = personD
    ind = str(int(lines[running_index].split(" ")[0])-1)
    vgg_ids[p].append(ind)
    filename = ind + "_" + str(p.split(".")[0]) + ".jpg"
    images[p].append(cv2.imread(path_database+filename, cv2.IMREAD_COLOR))
    

From the sources files, although the images downloaded were part of a curated data, several images need to be removed to be used in the context of this *educative* tutorial. The main reasons are:

*   too different from usual representation (make up, masks, ...)
*   really poor image quality
*   irrelevance and/or error in dataset
*   same image already in dataset
*   cropped image

Considering the tight selection of images to train our model (20 from personA and 20 from personB), and the relatively large global amount of image candidates (1000 for each person), it is acceptable to reject the images we know won't help.

From the initial retrieved images, we then remove the undesired images, that we copy in discard images folder, for tracking purposes. We/you may want to use them later.

---
`datasets_size`: dictionary of the size required per persons ( keys = person names)

`to_remove`: dictionary containing the indices to remove, per persons ( keys = person names)

`print_images = False` indicates that the remaining images in `images` dictionary will not be printed. 

---

From the remaining images (after rejection), we can apply randomly select the images that are part of the final sets (training and test sets not split yet).

In [0]:
# Dictionary of the size required (see section 3)
datasets_size = {}
datasets_size[personA] = 30
datasets_size[personB] = 30
datasets_size[personC] = 10
datasets_size[personD] = 10


# manually remove images that are not relevant or considered not good enough to be part of the dataset
to_remove = {}
to_remove[personA] = [0,1,4,8,12,13,16,23,28,34,36,42,44,47,48,49,54]
to_remove[personB] = [4,7,11,12,13,16,21,22,23,24,25,26,27,32,36,39,41,46,49,53,55,58]
to_remove[personC] = [0,1,6,7,8,11,14,16,17,19,20,21,24]
to_remove[personD] = [0,3,5,6,8,10,12,15,16,17,24]

# goal is to sort in descending to remove elements from lists without modifying the indexes
for p in persons:
    to_remove[p].sort(reverse = True)


# retrieve images candidates
# --------------------------
if len(os.listdir(path_datasets) ) == 0 or True:
    logging.debug("datasets empty - need to retrieve all !")
    # removing images to discard
    for person in persons:
        for index in to_remove[person]:
            img = images[person].pop(index)
            logging.debug("Removing item " + str(index) + " from list " + str(person))
            try:
                filename = path_discard +  str(index) +"_discarded_" + str(person.split(".")[0]) + ".jpg"
                cv2.imwrite(filename, img) 
            except:
                logging.error("Error while writing discarded image " + str(filename))

    # randomly select among remaining images
    for person in persons:
        # build list of indices from remaining images
        logging.debug("Phase 2 (part 1) -> random indices selection for " + str(person))

        images_ = []
        indices = []
        new_ids = []
        # prev_index = []
        random.seed(person)

        while len(indices) < datasets_size[person]:       
            index = random.randrange(0, len(images[person]))
            if index in indices:
                logging.debug("Index among remaining = " + str(index) + " => already there")
                continue
            else:
                # prev_index.append(index)
                indices.append(index)

        logging.debug("Phase 2 (part 2) -> image selection based on indices")

        for index in indices:
            img = images[person][index]
            images_.append(img)
            filename = path_datasets +  str(vgg_ids[person][index]) + "_" + str(person.split(".")[0]) + ".jpg"
            logging.debug("saved: " + str(filename))
            cv2.imwrite(filename, img) 
            new_ids.append(vgg_ids[person][index])
        images[person] = images_
        vgg_ids[person] = new_ids
else:
    logging.debug("folders not empty => can build directly images dictionnary")

# logging.debug("Number of images keys=" + len(images.keys))
# logging.debug("Number of images values=" + len(images.values))

logging.info(pretty_return_dict_size(images))
""" 
print images to get to_remove indices
"""
print_images = False
if print_images:
    for person in persons:
        counter = 0
        for img in images[person]:
            h = 0
            w = 0
            img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            faces = faceCascade.detectMultiScale(
                img_gray,
                scaleFactor=1.13,
                minNeighbors=10,
                minSize=(30, 30),
                flags=cv2.CASCADE_SCALE_IMAGE
            )
            for (x,y,w,h) in faces:
                # faces_cropped[person].append(img[y:y+h, x:x+w])
                cv2.rectangle(img, (x, y), (x+w, y+h), (0, 255, 0), 2)
            logging.debug("------------------------------------------------------")
            logging.debug("Photo ID = " + str(counter))
            logging.debug("size = " + str((h,w)))
            cv2_imshow(img)
            counter += 1

Mount drive and save images, according to the parameter `to_drive_confirmation` value. *Change at your own risk ;-)*

In [0]:
# Save to drive folders
path_drive_DB = r"/content/drive/My Drive/ComputerVision/DATABASE"
path_drive_Datasets = r"/content/drive/My Drive/ComputerVision/DATASETS"

# drive folders should be properly set up

if to_drive_confirmation:
    
    logging.warning("You need to have a drive mounted for this snippet to run successfully")

    try:
        drive.mount('/content/drive')
    except:
        pass
    try:
        shutil.rmtree(path_drive_DB)
        shutil.rmtree(path_drive_Datasets)
    except:
        logging.error("Error in rmtree") 


    try: 
        os.mkdir(path_drive_DB) 
        os.mkdir(path_drive_Datasets)
    except OSError as error: 
        logging.error(error) 
    
    logging.debug("Saving database in drive : start")

    fromDirectory = path_database
    toDirectory = path_drive_DB
    copy_tree(fromDirectory, toDirectory)

    logging.debug("Saving datasets in drive : start")

    fromDirectory = path_datasets
    toDirectory = path_drive_Datasets
    copy_tree(fromDirectory, toDirectory)

    logging.debug("Saving: done !")


In [0]:
path_vgg = r"/content/sample_data/CV__Group_assignment"
path_drive_vgg = r"/content/drive/My Drive/ComputerVision/CV__Group_assignment"

# drive folders should be properly set up

if to_drive_confirmation_vgg:
    
    logging.warning("You need to have a drive mounted for this snippet to run successfully")

    try:
        drive.mount('/content/drive')
    except:
        pass
    try:
        shutil.rmtree(path_drive_vgg)
    except:
        logging.error("Error in rmtree") 

    try: 
        os.mkdir(path_drive_vgg) 
    except OSError as error: 
        logging.error(error) 
    
    logging.debug("Saving database in drive : start")

    fromDirectory = path_vgg
    toDirectory = path_drive_vgg
    copy_tree(fromDirectory, toDirectory)

    logging.debug("Saving: done !")


From the raw images saved in the `images` dictionary, the faces are extracted using the *HaarCascade* method.

The following code is based on the tutorial: [How to detect faces using Haar Cascade](https://www.digitalocean.com/community/tutorials/how-to-detect-and-extract-faces-from-an-image-with-opencv-and-python)

The faces are saved in a new dictionary: `faces_cropped`

In [0]:
faceCascade = cv2.CascadeClassifier(os.path.join(base_path, "haarcascade_frontalface_default.xml"))
faces_cropped = {}

with open(file_info, 'r') as f: 
    lines = f.readlines()

for person in persons:

    faces_cropped[person] = []

    for img in images[person]:
        img_ = img.copy()
        img_gray = cv2.cvtColor(img_, cv2.COLOR_BGR2GRAY)
        faces = faceCascade.detectMultiScale(
            img_gray,
            scaleFactor=1.13,
            minNeighbors=10,
            minSize=(30, 30),
            flags=cv2.CASCADE_SCALE_IMAGE
        )
        for (x,y,w,h) in faces:
            faces_cropped[person].append(img[y:y+h, x:x+w])
            cv2.rectangle(img_, (x, y), (x+w, y+h), (0, 255, 0), 2)

        # h, w = img_.shape[:2]
        # draw_box(lines, int(vgg_ids[person][running_index])+1, img_, person)
        # cv2_imshow(cv2.resize(img_, (w // 2, h // 2)))

logging.info("Faces extracted and saved in dictionnary faces_cropped")
logging.info(pretty_return_dict_size(faces_cropped))

At this point, `faces_cropped` dictionary contains 30 cropped faces for personA and B, and 10 images for personC and personD.

The following code selects randomly (based on a seed) the 20 images part of the training set for personA and personB. The other faces (10 for each person) are then part of the test set.

---

* `training_set`: dictionary containing faces cropped (original size) part of the training set
* `test_set`: dictionary containing faces cropped (original size) part of the test set
---

At this point, there is not (yet) dedicated validation sets. It is discussed later on.

All the training will be done on the training set faces, without any tailoring or dedicated fitting on the test set images. Indeed, metrics on the test set faces indicate how well our model will generalize. It's therefore important to not influence our model with the data of the test set.


In [0]:
training_sets_size = {}
training_sets_size[personA] = 20
training_sets_size[personB] = 20
training_sets_size[personC] = 0
training_sets_size[personD] = 0

test_sets_size = {}
test_sets_size[personA] = 10
test_sets_size[personB] = 10
test_sets_size[personC] = 10
test_sets_size[personD] = 10

training_set = {}
test_set = {}
for person in persons:
    image_ = faces_cropped[person]
    training_set_ = []
    random.seed(person)
    init_set = set(range(0, len(image_)))

    indices_training = random.sample(init_set, training_sets_size[person])
    indices_test = list(init_set - set(indices_training))

    training_set[person] = [faces_cropped[person][i] for i in indices_training] 
    test_set[person] = [faces_cropped[person][i] for i in indices_test]

logging.info("Faces saved in dictionnary training_set: ")
logging.info(pretty_return_dict_size(training_set))
logging.info("Faces saved in dictionnary test_set: ")
logging.info(pretty_return_dict_size(test_set))

In [0]:
# show_images_from_dict(training_set)

In [0]:
# show_images_from_dict(test_set)

### Faces of the training set
- 20 faces of Emma Stone, personA
- 20 faces of Bradley Cooper, personB

PersonA and PersonB are quite different, A being a female, and B a male. Furthermore, the images within a class are somehow dissimilar as well
- different viewpoints (front, left, right)
- different lightening conditions
- not same hair color
- beard/no beard (personB)
- not same (limited) background

However, a similar characteristic is that both of the actors are most of the time smiling on the faces extracted. Sometimes showing their teeth, sometimes not.


In [0]:
#  Visualisation 
training_set_matrix = get_matrix_from_set(training_set, color = True, sq_size = sq_size,flatten = True)
plot_matrix(training_set_matrix, color = True, my_color_map = plt.cm.viridis, h=4, w=10, transpose = False)

### Faces of the test set
Test images are needed for four persons:
- 10 faces of Emma Stone, personA
- 10 faces of Bradley Cooper, personB
- 10 faces of Jane Levy, personC
- 10 faces of Marc Blucas, personD

(A - C) and (B - D) respectively share some characteristics:
* both female / male
* same kind of skin tone
* visually quite similar (especially for A and C)

Within each groups, as for the training set, the faces are taken from different viewpoints, lightening conditions, ...


In [0]:
test_set_matrix = get_matrix_from_set(test_set, color = True, sq_size=sq_size, flatten = True)
plot_matrix(test_set_matrix, color = True, my_color_map = plt.cm.viridis, h=4, w=10, transpose = False)

# Feature Representations

A feature representation of an object is intuitively a piece of *information*, of a reduced dimension with respect to the object, and that captures the object.
It tells what defines the object, and allows differentiating different objects.

In the context of an image, a good feature needs to be:
- **robust**: the same feature extracted from the same object on an image should be *close*, even if the lightening condition change, the view point change, ...
- **discriminative**: different images, representing different object, should lead to different representation in the feature space. As a toy example, the size of an image is not a good feature to detect a person, as several person can be represented in images of the same size.
Two types 

We will look at two features:
1. **HOG**: Histogram of oriented gradients. This is a handcrafted feature, extracted using a specific algorithm 
2. **PCA**: Principal Component Analysis. This is a feature learnt from the data.






## Histogram of Oriented Gradients - HOG



As this is a tutorial of Computer Vision, let's look first at what is visually / intuitively the HOG on a real image - one of the Emma Stone (personA) faces. The execution of the following code snippet shows on the left the input face, and on the right, the results.

Then, we'll see the details of the algorithm, and its specificities (parameters)

This section is extensively inspired by [this course](https://www.learnopencv.com/histogram-of-oriented-gradients/), while the technique has been introduced by [this paper](http://lear.inrialpes.fr/people/triggs/pubs/Dalal-cvpr05.pdf), which I strongly advise to read!

In [0]:
"""
First example - Emma stone first image
"""
src = faces_cropped[personA][0]
# cv2_imshow(src)

#1 resizing
resized_img = cv2.resize(src, (sq_size, sq_size))
#2 computing HOG
fd, hog_image = skimage_feature_hog(resized_img, 
                    orientations=9, 
                    pixels_per_cell=(8,8), 
                    cells_per_block=(2, 2), 
                    block_norm = "L2",
                    visualize=True, 
                    transform_sqrt = True,
                    multichannel=True)

"""
Plotting results
"""

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 8), sharex=True, sharey=True) 
ax1.imshow(cv2.cvtColor(resized_img, cv2.COLOR_BGR2RGB))
ax1.set_title('Input image') 

# Rescale histogram for better display 
hog_image_rescaled = exposure.rescale_intensity(hog_image, in_range=(0, 10)) 

ax2.imshow(hog_image_rescaled, cmap=plt.cm.gray) 
ax2.set_title('Histogram of Oriented Gradients - rescaled')
logging.debug("HOG Rescaled: " + str(hog_image_rescaled.min()) + " -> " + str(hog_image_rescaled.max()) )

# ax3.imshow(hog_image, cmap=plt.cm.gray) 
# ax3.set_title('Histogram of Oriented Gradients')
# logging.debug("HOG: " + str(hog_image.min()) + " -> " + str(hog_image.max()) )
plt.show()

# logging.debug(hog_image_rescaled.shape)
logging.debug("fd shape = " + str(fd.shape))

### HOG - What is that ?

HOG is a feature descriptor that extracts information from an image (or more precisely, a patch) based on the gradients in this image. More specifically, it builds a vector representing the weighted distribution of the gradients orientation across the images.

#### Why is it interesting ?
Let's remember that our goal is to perform image classification and identification. 
A face can be recognized through the inherent shapes: circular of face, shapes of the eyes, the nose, potentially the glasses, etc. The *edge* information is therefore useful! It is even more useful than the colors... Intuitively, you can think about recognizing someone familiar with only some contours of one face.

![Drawing Obama](http://www.drawingskill.com/wp-content/uploads/2/Barack-Obama-Drawing-Pics.jpg)

It is easy to recognize the former US President, while no color information is given. Intuitively, HOG gives the same information:
- magnitude of gradient is large around the edges and corners
- orientation gives the shape

#### How to compute the HOG of an image ?

In a nutshell:
- The gradients are first computed on each pixel. 
- The gradients orientation and magnitude are used to build an histogram for a cell. The size of a cell is typically 8 x 8 pixels. 
- Those histogram are normalized 
- All the histograms computed on the images are then concatenated in a *long* vector, yet much smaller than original image. 

In the upcoming sections, we will detail the process, as well as the code and parameters required at each step.

The following code snippet is a homemade class required to compute the HOG. The results obtained with this code will be compared with the infamous skimage library optimized for the HOG descriptor. 

You can simply run the snippet and come back later on to see the details of the implementation.


In [0]:
class MyHog():
    
    def __init__(self, img):
        self.img = img # image of the size 64x64; 64x128; 128x128; ... => resized image of the original
        self.mag_max, self.orn_max = self.compute_gradients()

    def compute_gradients(self):
        gx = cv2.Sobel(self.img, cv2.CV_32F, 1, 0, ksize = 1)
        gy = cv2.Sobel(self.img, cv2.CV_32F, 0, 1, ksize = 1)

        mag, angle = cv2.cartToPolar(gx, gy, angleInDegrees=True)
        orn = angle.copy()

        logging.debug("mag shape :" + str(mag.shape))
        logging.debug("orn shape :" + str(orn.shape))

        # constructing matrices of max dimension
        mag_max = np.zeros((mag.shape[0], mag.shape[1]))
        orn_max = np.zeros((orn.shape[0], orn.shape[1]))
        for i in range(mag.shape[0]):
            for j in range(mag.shape[1]):
                mag_max[i,j] = mag[i,j].max()
                idx = np.argmax(mag[i,j])
                orn_max[i,j] = orn[i,j,idx] 

        # mag_max = mag_max.T    
        # orn_max = orn_max.T 

        return mag_max, orn_max

    def get_cells_mag_orn(self, y_start, x_start, cell_h, cell_w):
        """
        returns the cell magnitude, orientation and "clipped" orientation 
        ( where 0 -> 360 is mapped into 0 -> 180)
        """
        cell_mag = np.zeros((cell_h,cell_w))
        cell_orn = np.zeros((cell_h,cell_w))
        for i in range(cell_h):
            for j in range(cell_w):
                cell_mag[i,j] = self.mag_max[y_start+i, x_start+j]
                cell_orn[i,j] = round(self.orn_max[y_start+i,x_start+j])
        
        cell_orn_clipped = cell_orn.copy() 
        cell_orn_clipped = ((cell_orn_clipped) + 90 ) % 360
        for i in range(cell_h):
            for j in range(cell_w):
                if 0 <= cell_orn_clipped[i,j] < 180:
                    cell_orn_clipped[i,j] = 180 - cell_orn_clipped[i,j]
                elif 180 <= cell_orn_clipped[i,j] <=360:
                    cell_orn_clipped[i,j] = 180 - cell_orn_clipped[i,j] % 180

        return cell_mag.T, cell_orn.T, cell_orn_clipped.T
    
    def fill_bins_one_pixel(self, mag, orn, bin_list, implementation_type = "skimage"):
        """
        # mag: magnitude of the gradient of 1 px
        # orn: orientation of the gradient of 1 px
        bin_list: reference, list of bins that is incremented
        """
        N_BUCKETS = len(bin_list)
        assert N_BUCKETS == 9, "N_BUCKETS is not 9!!!"
        size_bin = 20.
        if implementation_type == "learnopencv":
            if orn >= 160:
                left_bin = 8
                right_bin = 9
                left_val= mag * (right_bin * 20 - orn) / 20
                right_val = mag * (orn - left_bin * 20) / 20
                left_bin = 8
                right_bin = 0
            else:
                left_bin = int(orn / size_bin)
                right_bin = (int(orn / size_bin) + 1) % N_BUCKETS
                left_val= mag * (right_bin * 20 - orn) / 20
                right_val = mag * (orn - left_bin * 20) / 20
            
            assert left_val >= 0, "leftval = " + str(left_val) + ", " + str("mag = ") + str(mag) + " & orn = " + str(orn)
            assert right_val >= 0, "rightval = " + str(right_val) + ", " + str("mag = ") + str(mag) + " & orn = " + str(orn)

            # print(left_val)
            # print(right_val)

            bin_list[left_bin] += left_val
            bin_list[right_bin] += right_val

        elif implementation_type == "skimage":
            # easiest 
            """
            this implementation mimics the one from skimage
            """

            if 0 <= orn <= 10:
                bin_list[4] += mag
            elif 10 < orn <= 30:
                bin_list[3] += mag
            elif 30 < orn <= 50:
                bin_list[2] += mag
            elif 50 < orn <= 70:
                bin_list[1] += mag
            elif 70 < orn <= 90:
                bin_list[0] += mag
            elif 90 < orn <= 110:
                bin_list[8] += mag 
            elif 110 < orn <= 130:
                bin_list[7] += mag
            elif 130 < orn <= 150:
                bin_list[6] += mag
            elif 150 < orn <= 170:
                bin_list[5] += mag 
            elif 170 < orn <= 180:
                bin_list[4] += mag 
            else:
                raise RuntimeError("Impossible ! > " + str(orn))
        else:
            raise NotImplementedError



    def compute_hog_bins(self, y_start, x_start, cell_h, cell_w, show_src = True, show_results=True, figsize = (12,4)):
        """
        y_start: y value of the top left pixel
        x_start: x value of the top left pixel
        cell_h : height of the cell in which HOG is computed
        cell_w : width of the cell in which HOG is computed
        """
        cell_img = self.img[y_start:y_start + cell_h, x_start:x_start+cell_w]

        if show_src:
            tmp = self.img.copy()
            cv2.rectangle(tmp, (x_start-1, y_start-1), (x_start+cell_w+1, y_start+cell_h+1), (0,255,0))

            fig, ax = plt.subplots(1,1, figsize = (figsize[1],figsize[1]))
            ax.imshow(cv2.cvtColor(tmp, cv2.COLOR_BGR2RGB))
            ax.set_title("Selection of a cell")

            plt.show()

        # construction of the magnitude and orn matrices
        cell_mag, cell_orn, cell_orn_clipped = self.get_cells_mag_orn(y_start, x_start, cell_h, cell_w)

        number_of_bins = 9
        bin_list = np.zeros(number_of_bins)
        for i in range(cell_h):
            for j in range(cell_w):
                # m = round(mag_normalized[y_start+j,x_start+i].max())
                m = cell_mag[i,j]
                d = cell_orn_clipped[i,j]
                # print("m,d =" + str((m,d)))
                self.fill_bins_one_pixel(m,d,bin_list)
        
        # logging.debug("Bins computed:" + str(bin_list))
        n = np.linalg.norm(bin_list)
        bin_norms = bin_list/n
        # logging.debug("Bins normalized:" + str(bin_norms))
             
        if show_results:
            fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=figsize, sharex=True, sharey=True) 

            # ax1 => just to visually represent the arrows
            for i in range(cell_h):
                for j in range(cell_w):

                    radius = cell_mag[i,j] / (cell_mag.max() - cell_mag.min())
                    angle_ = cell_orn[i,j]
                    
                    orn_value_clipped = cell_orn_clipped[i,j]

                    mag_value = round(cell_mag[i,j])
                    
                    ax1.arrow(i, j, radius*np.cos(np.deg2rad(angle_)), radius*np.sin(np.deg2rad(angle_)), head_width=0.15, head_length=0.15, fc='b', ec='b')
                    ax2.text(i, j, str(orn_value_clipped.astype(np.int64)), fontsize=10,va='center', ha='center')
                    ax3.text(i, j, str(mag_value.astype(np.int64)), fontsize=10,va='center', ha='center')

            ax1.imshow(cv2.cvtColor(cell_img, cv2.COLOR_BGR2RGB))
            ax1.set_title('Input image') 

            ax2.matshow(cell_orn_clipped, alpha=0)
            ax2.set_title('Orientation values')

            ax3.set_title('Magnitude values')
            intersection_matrix = np.ones(cell_mag.shape)
            ax3.matshow(cell_mag, alpha = 0)
           
            plt.show()
        return bin_list, bin_norms

###HOG How-to, Step1: Preprocessing the image

Usually, the size of an image is not appropriate to perform the HOG computation. The easiest thing is just to resize the image to an appropriate size. In this tutorial, we use a multiple of 8 and a square image. Considering the smallest face cropped, we select 64 x 64 pixels.

Furthermore, as often in image processing, the largest the image, the more resource consuming it is. Keeping a reasonably small image helps in having a decent computation time.


In [0]:
logging.info("Toy Example")

img = faces_cropped[personA][0].copy()
resized_img = cv2.resize(img, (64,64))
logging.info("Shape of source  face: " + str(img.shape))
logging.info("Shape of resized face: " + str(resized_img.shape))


fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(8, 4), sharex=False, sharey=False) 

ax1.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
ax1.set_title('Input image') 

ax2.imshow(cv2.cvtColor(resized_img, cv2.COLOR_BGR2RGB))
ax2.set_title('Resized image') 
plt.show()

###HOG How-to, Step2: Computing the gradient for all pixels

Computing the gradient in horizontal ($x$) and vertical ($y$) directions can be done using a pass of the Sobel Filter, part of the *openCV* library. 
This is implemented in the `MyHog.compute_gradients()` function (see implementation of `MyHog` class, above. From these gradients in $x$ and $y$ we can derive the magnitudes and orientations in all pixels using the formulas:
> $
\begin{align} 
mag &= \sqrt{g_x^2 + g_y^2} \\ 
orn &= atan(\frac{g_y}{g_x}) 
\end{align}
$

This is implemented using *openCV* library with `cartToPolar`.

#####*Grayscale or Colored image*
> For grayscale image, every pixel has one value so that this computation is straightforward. For colored image, a pixel has three values (one for Red, one for Green, one for Blue). In the HOG algorithm, the gradients is computed for all three channels, and the final magnitude is the maximum of the three, and the orientation is the one corresponding to the magnitude.


In [0]:
"""
Create an object MyHog, which takes a resized image as input, and compute its 
gradients.
"""
myhog = MyHog(resized_img)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(8, 4), sharex=True, sharey=True) 

ax1.imshow(myhog.mag_max, cmap = plt.cm.jet)
ax1.set_title('Max of magnitude') 

ax2.imshow(myhog.orn_max, cmap = plt.cm.jet)
ax2.set_title('Max of Orientation') 
plt.show()

As visible on the magnitude and orientation plots above, only essential information regarding the edges is kept. 

###HOG How-to, Step3: Compute histograms for cells

The histograms are first computed for small cells. It has two major benefits
1. the representation is more compact: Suppose we take cells of $8 \times 8$ pixels. The gradient of each pixel is described using 2 numbers (magnitude and orientation), leading to 128 numbers. Considering an histogram applied on such a cell allows to represent those 128 numbers by a tiny array of typically 9 numbers. In total a colored image of $64 \times 64$ pixels is represented using $9*8*8$ vector.
2. the representation is less sensitive to noise, as applying a cell is equivalent to a low-pass filter. Higher frequency outliers are therefore of less importance.


The choice of the cell size is a design choice that can be modified. In a later section, we will modify this parameter to see how it can affect the results. 
In the paper that first presented the technique, a cell of $8 \times 8$ pixels was used - we will continue with this(hyper-)parameter. 


> *You may try yourself to modify the cell size or the x_start or y_start values, to see the influence on the histogram computed for that particular cell*





In [0]:
"""
Size of a cell
"""
cell_h = 8
cell_w = 8

"""
starting point of the cell on the image
"""
y_start = 3 * cell_h - 1
x_start = 2 * cell_w - 1

hog_val, hog_val_normalized = myhog.compute_hog_bins(y_start, x_start, cell_h, cell_w, show_results=True, figsize=(16,5))

fig, ax = plt.subplots(1,1, figsize = (2*5, 5))
ax.bar(["]70-90]","]50-70]", "]30-50]", "]10-30]","]10-\n180]","]150-\n170]","]130-\n150]","]110-\n130]", "]90-\n110]"], hog_val_normalized)
ax.set_title("Histogram computed with MyHog (homemade)")
plt.show()
logging.info("MyHog bins   normalized  : {}, {}, {}, {}, {}, {}, {}, {}, {}".format(*np.round(hog_val_normalized,2)))


**Legend of the above images**
1. source image with *cell* visible in flashy green,
2. details of the gradients computations
    - *leftmost*: cell enlarged with an arrow indicating the gradient: length of the arrow represent the magnitude, and orientation is the gradient orientation computed on that pixel
    - *middle*: matrix (shape == cell) containing the orientations computed (unsigned)
    - *rightmost*: matrix (shape == cell) containung the magnitude computed
3. histogram computed (`keyword = "skimage"`)


####In details

The details of building the histogram for a cell is not complicated:
- consider N bins. N is a design parameter, and each of the bins represent a range of gradient orientations. I have chosen N = 9, following the [introducing paper](http://lear.inrialpes.fr/people/triggs/pubs/Dalal-cvpr05.pdf), as it induces a granularity fine enough to observe change in the picture.
> The HOG is usually applied using **unsigned gradients**. The numbers on the orientation matrix are between 0 and 180 instead of 0 and 360 degrees. Concretely, an angle $\alpha [deg] $ and $(180 + \alpha) [deg]$ contribute to the same bin. Empirically, it has been observed that it wasn't decreasing performance in the detection. Of course, nothing forbids to use signed gradients. 
- for a particular pixel:
    - the bin is selected according to the orientation of the gradient; 
    - the value that goes in the bin is based on the magnitude. Different methods were proposed by the [introducing paper](http://lear.inrialpes.fr/people/triggs/pubs/Dalal-cvpr05.pdf), and are also explained in details for instance in [vidha blog](https://www.analyticsvidhya.com/blog/2019/09/feature-engineering-images-introduction-hog-feature-descriptor/). The class `MyHog` above contains two implementation: either the magnitude is split proportionnaly between two bins (as described in [learnopencv](https://www.learnopencv.com/histogram-of-oriented-gradients/), or -- as done in *openCV* library --, the whole magnitude is assigned to the closest bin. 

While this step is not difficult, let's realize in image how it's done!


####Creation of dedicated images

The following function allows creating images "on demand", in order to better understand the histogram computation. The parameter "special" indicate what type of image is required.

In [0]:
"""
CREATE_IMAGE returns an image created based on a keyword. 
"""
def create_image(height, width, special=None):

    mat0 = np.ones((height, width), dtype=np.uint8)*255
    mat1 = np.ones((height, width), dtype=np.uint8)*255
    mat2 = np.ones((height, width), dtype=np.uint8)*255

    if special == "center_black":
        mat0[height//2-1:height//2+1, width//2-1 : width//2+1 ] = 0
        mat1[height//2-1:height//2+1, width//2-1 : width//2+1 ] = 0
        mat2[height//2-1:height//2+1, width//2-1 : width//2+1 ] = 0
    elif special == "center_gray":
        mat0[height//2-3:height//2+3, width//2-3 : width//2+3 ] = 125
        mat1[height//2-3:height//2+3, width//2-3 : width//2+3 ] = 125
        mat2[height//2-3:height//2+3, width//2-3 : width//2+3 ] = 125
        mat0[height//2-1:height//2+1, width//2-1 : width//2+1 ] = 0
        mat1[height//2-1:height//2+1, width//2-1 : width//2+1 ] = 0
        mat2[height//2-1:height//2+1, width//2-1 : width//2+1 ] = 0
    elif special == "90":
        mat0[0:height, width//2 : width ] = 0
        mat1[0:height, width//2 : width ] = 0
        mat2[0:height, width//2 : width ] = 0
    elif special == "180":
        mat0[height//2:height, 0 : width ] = 0
        mat1[height//2:height, 0 : width ] = 0
        mat2[height//2:height, 0 : width ] = 0
    elif special == "135":
        for i in range(height):
            for j in range(i,width):
                mat0[i,j] = mat1[i,j] = mat2[i,j] = 0
    elif special == "45":
        for i in range(height):
            for j in range(width-i-1,width):
                mat0[i,j] = mat1[i,j] = mat2[i,j] = 0
    elif special == "28_34_37":
        mat0[4, 6:8] = 200
        mat0[5, 4:8] = 150
        mat0[6, 2:8] = 100
        mat0[7, 0:8] = 50
        mat1 = mat0.copy()
        mat2 = mat0.copy()
    elif special == "up_01":
        mat0[4, 4:8] = 250
        mat0[5, 0:8] = 50
        mat0[6, 0:8] = 50
        mat0[7, 0:8] = 50
        mat1 = mat0.copy()
        mat2 = mat0.copy()
    elif special == "up_10":
        mat0[4, 4:8] = 220
        mat0[5, 0:8] = 50
        mat0[6, 0:8] = 50
        mat0[7, 0:8] = 50
        mat1 = mat0.copy()
        mat2 = mat0.copy()
    elif special == "up_11":
        mat0[4, 4:8] = 225
        mat0[5, 0:8] = 50
        mat0[6, 0:8] = 50
        mat0[7, 0:8] = 50
        mat1 = mat0.copy()
        mat2 = mat0.copy()
    elif special == "up_15":
        mat0[4, 4:8] = 200
        mat0[5, 0:8] = 50
        mat0[6, 0:8] = 50
        mat0[7, 0:8] = 50
        mat1 = mat0.copy()
        mat2 = mat0.copy()
    elif special == "up_27":
        mat0[4, 4:8] = 150
        mat0[5, 0:8] = 50
        mat0[6, 0:8] = 50
        mat0[7, 0:8] = 50
        mat1 = mat0.copy()
        mat2 = mat0.copy()
    elif special == "up_152":
        mat0 = np.ones((height, width), dtype=np.uint8)*0
        mat1 = np.ones((height, width), dtype=np.uint8)*0
        mat2 = np.ones((height, width), dtype=np.uint8)*0
        mat0[0,6:7] = mat1[0,6:7] = mat2[0,6:7] = 255
        mat0[1,7] = mat1[1,7] = mat2[1,7] = 135
    elif special == "down_153":
        mat0 = np.ones((height, width), dtype=np.uint8)*255
        mat1 = np.ones((height, width), dtype=np.uint8)*255
        mat2 = np.ones((height, width), dtype=np.uint8)*255
        mat0[0,6:7] = mat1[0,6:7] = mat2[0,6:7] = 0
        mat0[1,7] = mat1[1,7] = mat2[1,7] = 125
    elif special == "up_141":
        mat0 = np.ones((height, width), dtype=np.uint8)*0
        mat1 = np.ones((height, width), dtype=np.uint8)*0
        mat2 = np.ones((height, width), dtype=np.uint8)*0
        mat0[0,6:7] = mat1[0,6:7] = mat2[0,6:7] = 255
        mat0[1,7] = mat1[1,7] = mat2[1,7] = 210
    elif special == "up_111":
        mat0 = np.ones((height, width), dtype=np.uint8)*0
        mat1 = np.ones((height, width), dtype=np.uint8)*0
        mat2 = np.ones((height, width), dtype=np.uint8)*0
        mat0[0,0:7] = mat1[0,0:7] = mat2[0,0:7] = 100
        mat0[1,7] = mat1[1,7] = mat2[1,7] = 255

    image = np.dstack((mat0, mat1, mat2))
    return image

####Examples of histogram computed on created images
In order to understand how the bins are fulled, let's look at a few of simple images. Those images are $8 \times 8$, meaning 1 cell == 1 image

* pure 90° gradient
* pure 180° gradient
* diagonal: 45°
* diagonal: 135°

For each case, we plot:
- 1) the arrows representing the gradients, 
- 2) the matrices of the magnitude and orientation values, 
- 3) the resulting histograms

**and** we log:

- the raw values of the histogram bins
- the normalized values of the histogram bins, using *L2-Normalization*:
$\begin{align}
bins\_values &= [x_1, x_2, x_3, x_4, x_5, x_6, x_7, x_8, x_9] \\
\|bins\_values\| &= \sqrt{x_1^2 + x_2^2 + x_3^2 + x_4^2 + x_5^2 + x_6^2 + x_7^2 + x_8^2 + x_9^2 } \\
bins\_values_{normalized} &=  \frac{v}{\|v\|} \\ 
&= \Bigg[ \frac{x_1}{\|v\|}, \frac{x_2}{\|v\|}, \frac{x_3}{\|v\|},  \frac{x_4}{\|v\|}, \frac{x_5}{\|v\|}, \frac{x_6}{\|v\|},  \frac{x_7}{\|v\|}, \frac{x_8}{\|v\|}, \frac{x_9}{\|v\|} \Bigg]
\end{align}$



#### Validation of intuition
To prove ourselves our implementation and understanding is correct, we will confront the results with the infamous library `skimage`, using `skimage.feature.hog` with the same parameters as the handmade function: 9 bins, an $8\times 8$ cell, and 1 cell per *block* (we discuss the *blocks* in the next section). Two parameters are still unknown: transform_sqrt and multichannel 
- `transform_sqrt`: if True, then the sqrt operator is applied to the global image first. It can give better results. We can safely leave it to False for the purpose of this exercices with the HOG bins...
- `multichannel`: simply indicates if the image is grayscale (`multichannel = False`) or in color (`multichannel = True`) 

<!--Note: we briefly discussed the `block_norm` parameter, but more to come in the next step.-->

####Finally...
Coming back to the very first example of the HOG, we saw the Emma Stone picture with weird white-ish pictograms describing her face... Well, thanks to the `skimage` library, it's very easy to get this image, and we show it for the toy example we are studying now. It allows grabbing the full overview of how, eventually, the complete histogram and visualization is computed.

> *Of course, don't hesitate to change yourself the list of images that are analyzed, considering the list implemented. You find the keywords accepted in the special list*


In [0]:
"""
Creation of the images
"""
special = ["up_01","45","90", "135","180","up_10","up_11","up_15", "up_27", "28_34_37", "up_111", "up_141", "up_152", "down_153","center_black", "center_gray"]
list_as_example = ["90", "180", "45", "135", "28_34_37"]

# created_img = create_image(cell_h,cell_w, "45")

"""
Homemade implementation of the histogram
and
Validation with an optimized library
"""
for keyword in list_as_example:
    logging.info("Considering image: " + str(keyword))

    # creation of the simple image
    created_img = create_image(cell_h, cell_w, keyword)

    # creation of MyHog object
    toyhog=MyHog(created_img)

    # compute bins using MyHog
    y_start_loc = 0
    x_start_loc = 0
    bins, bins_normalized = toyhog.compute_hog_bins(y_start_loc, x_start_loc, cell_h, cell_w, show_src=False, show_results=True, figsize = (14,4))

    # compute bins using Skimage 
    fd, hog_image = skimage_feature_hog(created_img, 
                orientations=9, 
                pixels_per_cell=(8,8), 
                cells_per_block=(1, 1), 
                block_norm = "L2",
                visualize=True, 
                transform_sqrt = False,
                multichannel=True)

    # plot results from Skimage
    plt.figure()
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 4), sharex=False, sharey=False) 

    # Rescale hog for better display 
    hog_image_rescaled = exposure.rescale_intensity(hog_image, in_range=(0, 10)) 


    xlabels = ["]70-90]","]50-70]", "]30-50]", "]10-30]","]10-\n180]","]150-\n170]","]130-\n150]","]110-\n130]", "]90-\n110]"]
    x = np.arange(9)
    w = 0.2
    ax1.bar( x-w, bins_normalized,  width=2*w, align='center',color="b")
    ax1.bar( x+w, fd, width=2*w, align='center',color="r")
   
    ax1.set_title("Histogram computed by Skimage.feature.hog")
    ax1.legend(["MyHog (homemade)", "Skimage.feature.hog"])
    # start, end = ax.get_xlim()
    # ax.xaxis.set_ticks(np.arange(start, end, 1))
    # ax1.set_xticklabels(xlabels)
    ax1.set_xticks(x)
    ax1.set_xticklabels(xlabels)

    ax2.imshow(hog_image_rescaled, cmap=plt.cm.gray) 
    ax2.set_title('Histogram of Oriented Gradients - visual')
    plt.show()



    logging.info("MyHog bins     computed  : {}, {}, {}, {}, {}, {}, {}, {}, {}".format(*np.round(bins,2)))
    logging.info("MyHog bins   normalized  : {}, {}, {}, {}, {}, {}, {}, {}, {}".format(*np.round(bins_normalized,2)))
    logging.info("Skimage bins normalized  : {}, {}, {}, {}, {}, {}, {}, {}, {}".format(*np.round(fd,2)))
    logging.info("***"*30)





> Notice: as a reminder, the purpose of the `MyHog` class (or any of the other class from this tutorial) is definitely not to reproduce exactly the behavior of a well-known and optimized library, but solely to break the magic behind using a library function without understanding the algorithm behind. As a result, the HOG computed may differ in several ways.

####Coming back to our initial cell...
Emma Stone cell defined above can now be shown in terms of HOG, helped by the `skimage` library.



In [0]:
"""
Retrieving the cell defined above
"""
cell_img=resized_img[y_start:y_start + cell_h, x_start:x_start+cell_w]

"""
computing HOG of the cell using same parameters
"""
fd, hog_image = skimage_feature_hog(cell_img, 
                    orientations=9, 
                    pixels_per_cell=(8,8), 
                    cells_per_block=(1, 1), 
                    block_norm = "L2",
                    visualize=True, 
                    transform_sqrt = False,
                    multichannel=True)



plt.figure()

fig, (ax0, ax1, ax2) = plt.subplots(1, 3, figsize=(12, 4), sharex=False, sharey=False) 
# hog_cropped = hog_image[y_start:y_start + cell_h, x_start:x_start+cell_w]
ax0.imshow(cv2.cvtColor(resized_img, cv2.COLOR_BGR2RGB))
ax0.set_title('Input image') 

ax1.imshow(cv2.cvtColor(cell_img, cv2.COLOR_BGR2RGB))
ax1.set_title('Extracted cell') 

# Rescale histogram for better display 
hog_image_rescaled = exposure.rescale_intensity(hog_image, in_range=(0, 10)) 
ax2.imshow(hog_image_rescaled, cmap=plt.cm.gray) 
ax2.set_title('Histogram of Oriented Gradients - rescaled')
# print(hog_image_rescaled)
plt.show()

logging.info("Skimage bins normalized  : {}, {}, {}, {}, {}, {}, {}, {}, {}".format(*np.round(fd,2)))




This is the end of the Step3: the computation of the histogram for one cell! A careful eye will have seen the parameters `cells_per_block` and `block_norm` of the library method. This is linked to Step4: Block Normalization!


###HOG How-to, Step4: Block normalization

In the Step3, we have extensively seen how to compute the histogram of gradients for a cell. We are almost at the end of the feature representation build up, but there are yet one normalization step. 
> In the previous step, we actually already normalized the histogram values using *L2-Normalization*. This is a simple case of the Block normalization where there is 1 cell per block. In general, we can define to perform Block normalization on more than 1 block. A common value is 4, as discussed in the [introducing paper](http://lear.inrialpes.fr/people/triggs/pubs/Dalal-cvpr05.pdf). 

####<u>Why do we need normalization ?</u>
 
When normalized, a histogram becomes independant from the lighting variation. 
Indeed, illumination has the impact of increasing/decreasing the values of the pixels in a cell. 

Using normalization, a change on the pixel value has no impact if all the pixels in a cell are subject to the same change. 
> let's say a low illumination make the pixel values divided by two. Having a normalized histogram on the cell will not be affected by such a change, as in the end, the absolute value is not important: only the relative value of one pixel to others matter. This is the very essence of the normalization.

As a result, normalizing the histogram makes it quite independant of the lighting condition (provided that in a cell, all the pixels have the same lighting condition, which seems a sensible assumption).

####<u>Normalizing by block</u> 
A nice visualization of the normalization by block of multiple cells is given in [learnopencv](https://www.learnopencv.com/wp-content/uploads/2016/12/hog-16x16-block-normalization.gif) where we see in green the different cells, and in blue a block of 4 cells. 
Using a block normalization - so, normalizing multiple cells at ones, and slide the normalization window across the image - is introduced in the [introducing paper](http://lear.inrialpes.fr/people/triggs/pubs/Dalal-cvpr05.pdf) which shows some benefits in terms of missing rate. Typically, 4 cells per blocks is often used. In a later section (see Classification), a grid search tends to try out other block sizes.

####<u>What normalization ?</u>
Several normalization can be considered: *L1*, *L2*, *L2-Hys*, ...



In [0]:
"""
Definition of the block size (in number of px)
1 cell = 8 x 8
1 block => 16 x 16 => 4 cells
"""
block_w = 16
block_h = 16

x_cells = np.arange(0,64,8)
y_cells = np.arange(0,64,8)

# credit: https://stackoverflow.com/questions/44816682/drawing-grid-lines-across-the-image-uisng-opencv-python
def draw_grid(img, line_color=(0, 255, 0), thickness=1, type_=8, pxstep=8):
    '''(ndarray, 3-tuple, int, int) -> void
    draw gridlines on img
    line_color:
        BGR representation of colour
    thickness:
        line thickness
    type:
        8, 4 or cv2.LINE_AA
    pxstep:
        grid line frequency in pixels
    '''
    x = pxstep
    y = pxstep
    while x < img.shape[1]:
        cv2.line(img, (x, 0), (x, img.shape[0]), color=line_color, lineType=type_, thickness=thickness)
        x += pxstep

    while y < img.shape[0]:
        cv2.line(img, (0, y), (img.shape[1], y), color=line_color, lineType=type_, thickness=thickness)
        y += pxstep

def draw_one_block(img, origin=(0,0), line_color=(255,0, 0), block_size = 16, thickness=1, type_=8):
    cv2.rectangle(img, origin, (origin[0]+block_size, origin[1]+block_size), line_color, thickness=thickness, lineType =type_)

def draw_all_blocks(img, thickness):
    color_list = [(255,0,0), (255,255,0), (255,0,255)]
    x = 0
    y = 0
    counter = 0
    while x < img.shape[1]-8:
        # cv2.line(img, (x, 0), (x, img.shape[0]), color=line_color, lineType=type_, thickness=thickness)
        # draw_one_block(img, (x,y))
        
        
        while y < img.shape[0]-8:
            # cv2.line(img, (0, y), (img.shape[1], y), color=line_color, lineType=type_, thickness=thickness)
            lc = color_list[counter%3]
            draw_one_block(img, (x,y),line_color=lc, thickness=thickness)
            counter += 1
            y += 8
        y=0
        x += 8
    return counter

"""
creating a green grid covering the resized image
"""

grid_cells_img = resized_img.copy()
draw_grid(grid_cells_img, type_=8)

"""
Creating the three first blocks
"""

first_block_img = grid_cells_img.copy()
second_block_img = grid_cells_img.copy()
third_block_img = grid_cells_img.copy()

draw_one_block(first_block_img, origin=(0,0), line_color=(255,0,0), thickness=2)
draw_one_block(second_block_img, origin=(8,0), line_color=(255,255,0),thickness=2)
draw_one_block(third_block_img, origin=(16,0), line_color=(255,0,255),thickness=2)

"""
Creating all the blocks on top of the cells
"""

blocks_img = grid_cells_img.copy()
counter = draw_all_blocks(blocks_img, thickness=1)

"""
Vizualization
"""

fig, (ax0, ax1, ax2, ax3) = plt.subplots(1, 4, figsize=(16, 4), sharex=False, sharey=False) 
ax0.imshow(cv2.cvtColor(grid_cells_img, cv2.COLOR_BGR2RGB))
ax0.set_title('Cells') 

ax1.imshow(cv2.cvtColor(first_block_img, cv2.COLOR_BGR2RGB))
ax1.set_title('first block') 

ax2.imshow(cv2.cvtColor(second_block_img, cv2.COLOR_BGR2RGB))
ax2.set_title('second block') 

ax3.imshow(cv2.cvtColor(third_block_img, cv2.COLOR_BGR2RGB))
ax3.set_title('third block') 

fig, ax = plt.subplots(1,1,figsize=(4,4))
ax.imshow(cv2.cvtColor(blocks_img, cv2.COLOR_BGR2RGB))
ax.set_title("All Blocks")

plt.show()

logging.info("In total, there are: " + str(counter) + " blocks possible in the picture")


As shown in the previous example, on the image chosen, there are $49$ blocks possible of size $(16 \times 16)$ pixels.

###HOG How-to, Step5: concatenation

After the normalization, the only step remaining is the concatenation of the computed vectors into a larger one, that represent the input image. This will be the feature representation of the image, based on the *oriented* gradients in that image.

###What size is this feature representation ?
- one cell is represented by $9$ numbers (histogram)
- four histograms are normalized together, leading to a $(36,1)$ vector
- there are $49$ such vectors representing the image.
    If the image as a width of size $(w*8)$ pixels, and a height of $(h*8)$ pixels, the image dimension is $(8*h \times  8*w)$. In such an image, they are :
    *   h cells vertically, and w cells horizontally,
    *  (h-1) blocks vertically and (w-1) blocks horizontally.

The length of the final vector is then $36 \cdot 49$ numbers, or a $(1764,1)$ vector.

Of course, this is still large... But much more compact that our initial $(64,64,3)$ array of $12288$ numbers



In [0]:
fd, hog_image = skimage_feature_hog(resized_img, 
                    orientations=9, 
                    pixels_per_cell=(8,8), 
                    cells_per_block=(2, 2), 
                    block_norm = "L2",
                    visualize=True, 
                    transform_sqrt = True,
                    multichannel=True)

"""
Plotting results
"""

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 8), sharex=True, sharey=True) 
ax1.imshow(cv2.cvtColor(resized_img, cv2.COLOR_BGR2RGB))
ax1.set_title('Input image') 

# Rescale histogram for better display 
hog_image_rescaled = exposure.rescale_intensity(hog_image, in_range=(0, 10)) 
ax2.imshow(hog_image_rescaled, cmap=plt.cm.gray) 
ax2.set_title('Histogram of Oriented Gradients - rescaled')
logging.debug("HOG Rescaled: " + str(hog_image_rescaled.min()) + " -> " + str(hog_image_rescaled.max()) )

plt.show()

# logging.debug(hog_image_rescaled.shape)
logging.info("Shape of the HOG feature : " + str(fd.shape))



<!--histogram construction is based on the gradient computed - both magnitude and orientation (as defined)

- the bin is selected according to the orientation (direction) of the gradient; 
- the value that goes in the bin is based on the magnitude

For instance on the toy example:
first pixel has:
* mag = 6; orn = 45°. So, the vote of this pixel goes for 75% in the bin of 40°, and 25% in the bin of 60°, as closer to 40°. As a result, we add 4.5 to bin nb 3, and 1.5 to bin nb 4. -->

###HOG: Exec summary

* one cell (typ 8 x 8) of an image is represented by a histogram 
    * the orientation and magnitude of the gradient are computed on each pixel
    * orientation of the gradient indicate a bin
    * magnitude indicate the amount to place into the bin
* the histogram is a vector of size 9 (as 9 bins)
* one block (16 x 16) histogram is the concatenation of the 4 histograms, each representing one cell of the block, hence represented by a (36 x 1) vector.
* the final HOG feature vector is based on the concatenation of all blocks.


For an image of 64 x 64, it follows:
* 8 cells along the width
* 8 cells along the height
* Number of cells = 64 cells
* Number of blocks = 7 * 7 = 49 blocks

Each block has a representative vector of dimension $(36 \times 1)$, and the resulting vector has dimension $(49*36 \times 1)$, or $(1764,)$


This ends the first part of Histogram of Oriented Gradient, where I showed in detail how to compute such a feature representation, and the meaning of the different parameters.

As many other parameters, the "hyper-parameters" of a HOG method should always be assessed according to a specific problem. 
- Insofar, the parameters have mainly be considered equal to their suggested value by the litterature, in this [HOG introduction](http://lear.inrialpes.fr/people/triggs/pubs/Dalal-cvpr05.pdf)
- Later, in this tutorial, we will optimize a classifier using a systematic method and a cross-validation set. 

Recall that we shall **never** use the test set to fine tune our model parameters.

<!--Still to do : 
 These [hog] parameters were obtained by experimentation and examining the accuracy of the classifier — you should expect to do this as well whenever you use the HOG descriptor. Running experiments and tuning the HOG parameters based on these parameters is a critical component in obtaining an accurate classifier.-->

### Detecting an object of interest in a new image

In this part, the goal is to use the HOG feature descriptor in order to detect if an object is present or not in a new image.

> we are not building a classifier or identificator (yet!) The goal is *just* to detect which region of an image have structures that correspond well to the ones of the feature descriptor.


To do so, the steps are:
1. Select an object
2. Compute its HOG feature description
3. Select a new image
4. pre-process this image in terms of dimensions
5. compute the image HOG at every location
6. Assess the matching between the descriptor and the image


####Select an object and compute its hog

As we have built a training set and a test set, let's just pick randomly one image of the training set. We can do even better and compute the HOG for all images in training set using the parameters already seen. We store the results in the dictionary `hog_training`. As *usual*, the keys are the persons names.

Later, we can select any of those as the `image_of_interest`


In [0]:
hog_training = {}

for person in persons:
    hog_training[person] = []
    for src_img in training_set[person]:
        resized_img = cv2.resize(src_img, (sq_size,sq_size))
        fd, hog_image = skimage_feature_hog(resized_img, 
                                            orientations=9, 
                                            pixels_per_cell=(8,8), 
                                            cells_per_block=(2, 2), 
                                            block_norm = "L2",
                                            visualize=True, 
                                            transform_sqrt = True,
                                            multichannel=True)
        hog_training[person].append((fd, hog_image, resized_img))

logging.debug(pretty_return_dict_size(hog_training))
logging.info("hog_training dictionary contains the HOG descriptors (resized) for all faces of the training set")

In [0]:
"""
Let's pick an image of interest, by its index [0 -> 19]
"""

idx_of_interest = 2
image_of_interest = training_set[personA][idx_of_interest]
hog_of_interest = hog_training[personA][idx_of_interest]

"""
Visualization of the image and its hog selected as image_of_interest
"""
fig, (ax0, ax1) = plt.subplots(1,2,figsize = (8,4), sharex=False, sharey=False)

ax0.imshow(cv2.cvtColor(image_of_interest, cv2.COLOR_BGR2RGB))
ax0.set_title("Image of interest from training set")

ax1.imshow(hog_of_interest[1])
ax1.set_title("Visualization of the HOG of interest")

logging.info("Shape of the descriptor       : " + str(hog_of_interest[0].shape))
logging.info("Shape of the descriptor (visu): " + str(hog_of_interest[1].shape))
logging.info("Shape of the image of interest: " + str(image_of_interest.shape))

####Select a new image
This is the image we want to apply the descriptor matching.
Put in another words, we want to verify, using a distance metric such as the euclidean distance, if the HOG descriptor of my *image_of_interest* presents similarities with a region in a new image.

Let's just pick a certain number of images from our **original** images downloaded, the ones that are not cropped yet, nor resized. 


In [0]:
person_ = personA # can be {personA, personB, personC, personD}
number_of_candidates = 3
random.seed("7/4/2020")
images_candidates = random.sample(images[person_], number_of_candidates)

fig = plt.figure(figsize=(12,4))
i=0

for img in images_candidates:
    ax=fig.add_subplot(1,number_of_candidates, i+1)
    ax.imshow( cv2.cvtColor(img, cv2.COLOR_BGR2RGB) )
    i+=1
plt.show()


The different images selected don't have the same size.

In [0]:
logging.info("Shapes of image on which to look for the image_of_interest: ")
for img in images_candidates:
    logging.info("Image shape: " + str(img.shape))

####Matching
In this larger step, we want to see if there is a match between the descriptor of interest and the image candidate. 

Several problems arise:
- The descriptor of interest has a fixed (designed) size of $(1764,1)$
- The different image candidates have different sizes,
- The image candidates contains more information than just a face

*Why not just cropping the faces on the image and resize it?*

We could of course do that - but that is not really the purpose :-) Rather, we want to assess, in **every location** of the image candidate, if there is a chance to see a pattern such as the descriptor provided of the image of interest.

*Simpler case*
Let's consider first that if the image candidate contains a face (=object), this face has approximately the same size as the original face of interest

One way to proceed is to slide, across the image candidate, a window of the size of the object to detect. 
- at *each* location, crop the part of the image candidate in the sliding window
> sliding at every each location is cumbersome and time consuming. The parameter `step`actually defines the amount of pixels that is skipped in both directions during the sliding.
- compute the HOG of this crop
- compare (using Euclidean distance for instance) this descriptor with the descriptor of the object to find
- go to the next location

At the end, the result is a score attributed to every location, indicating the correspondance between the object to detect, and the area in the image. 

 

*Helper functions*
- `get_local_crop`: realize the cropping before computing the HOG, ensuring the appropriate size to compute HOG
- `match_hog`: homemade matching between an object of interest and an image candidate, implementing this "sliding" accross the image
- `fill_matrix_min_neighbours`: compensate the effect of the `step`parameter in the plot

In [0]:
def get_local_crop(src_img, center_pixel, crop_shape, show = False):
    """
    src_img:
    center_pixel:
    crop_shape:
    """
    crop_height =  crop_shape[0]
    crop_width = crop_shape[1]
    top_= center_pixel[0] - crop_height // 2
    bottom_ = center_pixel[0] + crop_height // 2
    left_ = center_pixel[1] - crop_width // 2  
    right_ = center_pixel[1] + crop_width // 2
    if len(src_img.shape)> 2:
        crop = src_img[ top_ : bottom_, left_:right_, :]
    else:
        crop = src_img[ top_ : bottom_, left_:right_]

    if show:
        cv2_imshow(crop)
    return crop

def match_hog(src_img, hog_desc, original_face_shape, win_stride = (8,8), show = False):
    """
    src_img : image to analyse
    hog_desc: (fd, hog_image) of the corresponding faces_cropped
    original_face_shape: shape of the face used for hog_desc computation
    """
    height = src_img.shape[0] # height of the image to analyze
    width  = src_img.shape[1] # width of the image to analyze
    
    height_face = original_face_shape[0]
    width_face = original_face_shape[1]

    res_shape = (src_img.shape[0], src_img.shape[1])
    res = np.ones(res_shape)*-1

    if show:
        tmp_image = src_img.copy()
        cv2.rectangle(tmp_image, (width_face//2, height_face//2), (width - width_face//2, height - height_face//2), (0, 255, 0))
        cv2_imshow(tmp_image)

    running_h_idx = range(height_face //2, height - height_face//2+1, win_stride[0])
    running_w_idx = range(width_face //2, width - width_face//2+1, win_stride[1])

    for h_idx in running_h_idx:
        for w_idx in running_w_idx:
            local_crop = get_local_crop(src_img, (h_idx, w_idx), original_face_shape, False)
            local_resized_img = cv2.resize(local_crop, (64,64))
            local_fd = skimage_feature_hog(local_resized_img, 
                                            orientations=9, 
                                            pixels_per_cell=(8,8), 
                                            cells_per_block=(2, 2), 
                                            block_norm = "L2",
                                            visualize=False, 
                                            transform_sqrt = True,
                                            multichannel=True)

            # computing euclidean distance here !!            
            res[h_idx,w_idx]= np.linalg.norm(local_fd-hog_desc[0])
            if show:
                cv2_imshow(local_resized_img)

    return res

def fill_matrix_min_neighbours(matrx, win_stride = (16,16), margin = 0):
    """
    fill the gaps in the  computation by taking the min values from closest neighbours
    that were computed.
    """
    l = np.argwhere(matrx != -1)
    res = matrx.copy()

    if len(l)<2:
        idx_to_change = res == -1
        logging.warning("len < 2 --> len(idx_to_change) = " + str(len(idx_to_change)))
        res[idx_to_change] = res.max() + margin
        return res

    top_left_corner = l[0]
    bottom_right_corner = l[-1]

    for i in range(top_left_corner[0], bottom_right_corner[0]+win_stride[0]//2):
        for j in range(top_left_corner[1],bottom_right_corner[1]+win_stride[1]//2):
            if matrx[i,j] == -1:
                local_roi = get_local_crop(matrx, (i,j),win_stride)
                cand = local_roi[ local_roi != -1 ]
                res[i,j] = cand.min()
    idx_to_change = res == -1
    res[idx_to_change] = res.max() + margin
    return res

*Details*

For all the image in images_candidates list, 
- compute the matching on every required location (point spaced by win_stride)
- fill the non computed point with min neighbour
- normalized (L2) to get values between 0 -> 1
- show the results

In [0]:
for img in images_candidates:
    win_stride=(16,16)
    res = match_hog(img, hog_of_interest, image_of_interest.shape, win_stride)
    new_res = fill_matrix_min_neighbours(res, win_stride)

    logging.debug("worst match hog results: " + str(new_res.max()))
    logging.debug("best match hog results: " + str(new_res.min()))

    b = new_res.copy()
    bmax, bmin = b.max(), b.min()
    if bmax == bmin and bmax == 0:
        logging.info("Perfect match!")
    b = (b - bmin)/(bmax - bmin)
    # b = (b - bmin)/(bmax)

    logging.debug("worst match hog results normalized [expexted 1]: " + str(b.max()))
    logging.debug("best match hog results [expected 0]: " + str(b.min()))



    fig, (ax1, ax2) = plt.subplots(1,2 , figsize=(16, 8), sharex=True, sharey=True) 

    ax1.imshow(cv2.cvtColor(img,cv2.COLOR_BGR2RGB))
    ax1.set_title("Image where to find base face")

    # ax2.imshow(new_res, cmap="jet")
    # ax2.set_title("Results gaps filled with min neighbour")

    ax2.imshow(b, cmap="jet")
    ax2.set_title("Normalized results of Matching")

    plt.show()
    logging.debug("=====================================================")




####Matching - part2

As showed in the results above, it works ! Several observations nonetheless:
1. the borders are red (indicating  large distance): this is because of the sliding that considers the full object of interest is required in the image

2. It works even if not the exact same shape!
    * provided that a threshold is chosen, one could use this to recognize an object
    * only *LOCAL* description is given: only local object shape and appearances (to some extend) can be represented. 
    * Because of locality and limit of expressiveness, the face of Bradley Cooper and Emma Stone may look alike, in terms of HOG descriptors.

3. Only *ONE* size of the object is verified. If the object to find is of the same size as the object in the image, the descriptors will be very similar, and the distance small. However, if the object to find is smaller or larger in the image candidate, the HOG descriptor won't match at all. 
To solve this problem, one can perform multiscale detection, where the object is scaled several times to ensure to really detect the object, if it is present.

A nice overview of this multiscaling can be found in [pyimagesearch](https://www.pyimagesearch.com/2015/11/16/hog-detectmultiscale-parameters-explained/)

4. The detection of object with this sliding window takes time, and is even more time / resource consuming if used with multiscale analysis. Several techniques should be set in place (also discussed in [pyimagesearch](https://www.pyimagesearch.com/2015/11/16/hog-detectmultiscale-parameters-explained/)) such as:
    - reducing the size of the image candidate, without losing too much information
    - adjusting the HOG parameters so that the computation time is optimized for a **specific application**




###HOG Conclusion

In this first feature representation building, we studied in details how the descriptor is built and computed, and the different parameters that comes in play, and in particular
- the cell size
- the block size
- the normalization method

We have then used the HOG descriptor to try and detect if an object is in an image, by computing on (almost) every position of the image the descriptor and assessing the distance (as similarity metric) with the face of interest descriptor.
Doing so, we have then seen that the techniques works well to find region of a similar shape in the image, hence the locality of the descriptor. We also covered some of the challenges related to the images'size, locality of the descriptors, and complexity of the computations.


##  Principal Component Analysis - **PCA**

We have extensively covered HOG. Similarly, we will go through the PCA technique. First, we will give some intuition; then we will go through the maths, as this technique rely heavily on the computing, then we will apply PCA on the training set and try and observe some results.

When discussing PCA, we will mainly focus of the technique applied on images. There are plenty of blogs out there precisely defining and tailoring the techniques for all kinds of application. This tutorial is just an example of PCA applied to face images.




---

As for the previous HOG technique, homemade code is fully provided. This gives the details on the implementation and insight about how things are calculated. For this part of the tutorial, most of the examples are done with this homemade code. A section is dedicated to a demo of a library tool as well. For the classification and identification parts, however, library optimized code will be used as it's the purpose of those libraries. 
Don't hesitate to try out different parameters in the provided function, and please report any bug to geoffroy.herbin@student.kuleuven.be

---

- How to convert my image (you can work in color if you like) dataset to a 2D matrix?
- Can you exploit the dimensionality of this data matrix to make your computations more effective?
- Mean subtraction or not?
- Shall we use eigenvalue or singular value decomposition?
- How many non-zero eigenvalues/singular values should we have?

In [0]:
class MyPCA:
    """
    homemade class to perform PCA using several methods and compare
    Three methods are implemented
    - method = "svd": singular value decomposition technique
    - method = "eigen": nominal eigenvalue decomposition technique is implemented
    - method = "eigen_fast": eigenvalue decomposition using dimensionality 
                            reduction is implemented
    """
    def __init__(self, method = "svd"):
        self.method = method
        self.eigenvalues = None
        self.eigenvectors = None
        self.X_mean = None

    def fit(self, data):
        X = data.copy()
        n, m = X.shape
        assert n < m, "n is not smaller than m -> you most likely need " + \
                    "to transpose your input data"
        self.X_mean = np.mean(X, axis = 0)
        X -= self.X_mean

        self.eigenvalues = None
        self.eigenvectors = None

        if self.method == "svd":
            # singular value decomposition
            U, S, Vt = np.linalg.svd(X)
            self.eigenvalues = S**2 / (n - 1)
            self.eigenvectors = Vt.T[:,0:n]

        elif self.method == "eigen_fast":

            # compute small covariance matrix
            D = np.dot(X, X.T) / (n - 1)

            # eigen decomposition
            LD, W = np.linalg.eig(D)

            order_D = np.argsort(LD)[::-1]
            LD_sorted = LD[order_D]
            W_sorted = W[:,order_D]
            
            eigenVectors_sorted_tmp = np.dot(X.T, W_sorted)
            eigenVectors_sorted = np.empty(eigenVectors_sorted_tmp.shape)
            for i in range(n):
                v = eigenVectors_sorted_tmp[:,i]
                eigenVectors_sorted[:,i] = v / np.linalg.norm(v)
            
            self.eigenvalues = LD_sorted
            self.eigenvectors = eigenVectors_sorted
                
        elif self.method =="eigen":
            # compute covariance matrix

            Cov = np.dot(X.T, X) / (n - 1)

            # eigen decomposition
            LC, V =np.linalg.eig(Cov)

            # sort in appropriate order and keep only relevant component
            order = np.argsort(LC)[::-1]
            LC_sorted = LC[order][0:n]
            V_sorted = V[:,order][:,0:n]

            self.eigenvalues = LC_sorted
            self.eigenvectors = V_sorted.real
        else:
            raise RuntimeError("method value unknown")
        
    def projectPC(self, X, k):
        Vk = self.eigenvectors[:, 0:k]
        # logging.debug("Reduce X " + str(X.shape) + "using k = " + str(k) + " components")
        # logging.debug("Vk (4096 x k)= " + str(Vk.shape))
        X_reduced = np.dot(X, Vk)
        # logging.debug("X_reduced (n x k) = " + str(X_reduced.shape))

        return X_reduced
    
    def reconstruct(self, X_reduced, k, show = False):
        if len(X_reduced.shape) > 1:
            X_reduced = X_reduced[:,0:k]
        else:
            X_reduced = X_reduced[0:k]
        
        Vkt = self.eigenvectors[:, 0:k].T
        # logging.debug("Reduce using X_reduced = " + str(X_reduced.shape) )
        # logging.debug("Reconstruct using k = " + str(k) + " components")
        # logging.debug("Vkt (k x 4096)= " + str(Vkt.shape))
        X_hat_centered = np.dot(X_reduced, Vkt)
        # logging.debug("X_hat_centered.shape (20,4096): " + str(X_hat_centered.shape))
        if show:
            self.show_data(X_hat_centered, add_mean = True)
        return X_hat_centered 

    def compute_error(self, X, X_hat):
        return sqrt(mean_squared_error(X, X_hat))

    def show_principal_components(self,k, figsize=(10,4)):
        w = figsize[0]
        h = figsize[1]
        fig = plt.figure(figsize=(w,h)) 
        fig.subplots_adjust(left=0, right=1, bottom=0, top=1, hspace=0.05, wspace=0.05) 
        logging.debug("self.eigenvectors.shape = " + str(self.eigenvectors.shape) )
        for i in range(k):
            pc = self.eigenvectors[:,i]
            assert pc.shape[0]==(sq_size**2)*1 or pc.shape[0]==(sq_size**2)*3, "Not proper shape (expected (sq_size**2) (*3)) " + str(pc.shape)
            
            ax = fig.add_subplot(h, w, i+1, xticks=[], yticks=[]) 
            if color:
                pc_img = (np.reshape(pc.real, (sq_size, sq_size, 3))*255).astype("uint8") 
                ax.imshow(pc_img, interpolation='nearest') 
            else:
                pc_img = np.reshape(pc.real, (sq_size, sq_size))
                ax.imshow(pc_img  , cmap=plt.cm.gray, interpolation='nearest')

        plt.show()

    def compute_explained_variance(self, show=True):
        sum_all_eigenValues = sum(self.eigenvalues)
        logging.debug("\nSum of all eigenValues: " + str(sum_all_eigenValues))

        explained_variance      = [(value / sum_all_eigenValues)*100 for value in self.eigenvalues]
        cum_explained_variance  = np.cumsum(explained_variance)
        logging.debug("Cum explained variance : \n" + str(cum_explained_variance))
        if show:
            fig = plt.figure(figsize=(12, 6))
            ax1 = fig.add_subplot(121)
            ax1.bar(range(len(self.eigenvalues)), self.eigenvalues)
            ax1.set_xlabel('eigenvalues')
            ax1.set_ylabel('values')

            ax2 = fig.add_subplot(122)
            ax2.bar(range(len(explained_variance)), explained_variance)
            ax2.plot(range(len(cum_explained_variance)), cum_explained_variance, color='green', linestyle='dashed', marker='o', markersize=5)

            ax2.set_xlabel('eigenvalues')
            ax2.set_ylabel('% information ')
            ax2.legend( labels = ["Cumulative Expl. Var.", "Explained Variance"])
            ax2.grid()
            plt.show()
        return explained_variance, cum_explained_variance

    def show_data(self, X, add_mean = False):
        
        # copy so that adding the mean does not modify the original centered
        # data X
        X_ = X.copy()

        if len(X.shape) > 1:
            self._show_data(X_, add_mean)
        else:
            fig = plt.figure(figsize=(3,3))
            if add_mean:
                    X_ += self.X_mean
            # img = np.reshape(X_, (sq_size, sq_size))
            img = my_reshape(X_, sq_size, color)

            ax = fig.add_subplot(1, 1, 1, xticks=[], yticks=[]) 
            ax.imshow(img, cmap = my_color_map, interpolation='nearest') 

            plt.show()


    def _show_data(self, X, add_mean = False):
        fig = plt.figure(figsize=(8,8)) 
        fig.subplots_adjust(left=0, right=1, bottom=0, top=1, hspace=0.05, wspace=0.05) 
        i=0
        for x in X:
            if add_mean:
                    x += self.X_mean
            # assert x.shape[0]==(sq_size**2)*3, "Not proper shape (expected (sq_size**2)*3) " + str(x.shape)
            ax = fig.add_subplot(8, 5, i+1, xticks=[], yticks=[]) 
            img = my_reshape(x, sq_size, color) 
            ax.imshow(img, cmap = my_color_map, interpolation='nearest') 

            i+=1
        plt.show()

### Basic idea

If you are completely unfamiliar with the principal components analysis, the thread in [PCA intuition](https://stats.stackexchange.com/questions/2691/making-sense-of-principal-component-analysis-eigenvectors-eigenvalues) contains a wonderful layered explanation of what the PCA is, and what can its use be.
Another very nice introduction is given in [medium](https://medium.com/@aptrishu/understanding-principle-component-analysis-e32be0253ef0)

Essentially, the PCA technique extracts the information from the data to get the main directions intrinsically present in the data. To realize that, PCA uses the covariance, which is a *measure of the extent to which corresponding elements from two sets of ordered data move in the same direction* (definition extracted from [medium website](https://medium.com/@aptrishu/understanding-principle-component-analysis-e32be0253ef0))


Based on this covariance information, the PCA  *finds a new set of dimensions (or a set of basis of views) such that all the dimensions are orthogonal (and hence linearly independent) and ranked according to the variance of data along them. It means more important principle axis occurs first* (extracted from [medium website](https://medium.com/@aptrishu/understanding-principle-component-analysis-e32be0253ef0))
If you're more of a visual person, the plot [here](https://i.stack.imgur.com/lNHqt.gif) shows the first principal component of a cloud of points, with an animation that shows how the variance is minimized.

It is really an extraction of information from the data, hence an unsupervised technique, that is used to reduce the dimensionality of the problem. Applying PCA on images can therefore be used as a feature representation!






### PCA How-to, Step1: Pre-processing

The PCA is applied on *hyper-points*: points in our high-dimensional space. We reprensent each of those points by a vector. Applied on images, we then need to convert our sets of images into a useable format. 

The representation chosen is a $(n \times p)$ matrix where
- n is the number of images
- p is the dimension of the vector

The matrix containing the training data is then a $(40 \times p)$ matrix, and -- in our specific problem insofar -- the matrix containing the test data is a $(40 \times p)$ matrix.

#### Resizing
All the images (*hyper-points*) shall have the same dimensions to start with. There is therefore a first step of resizing all the images into a commonly appropriate size, keeping in mind that the faces are square images. The parameter indicating the lenght of this square side is defined as `sq_size` in this notebook. 

`sq_size = 64` indicates that the images are resized as a $(64 \times 64)$ matrix, containing 1 or 3 channels depending on their colorscale. 


#### Color or Grayscale ?
In the litterature, we find both implementation, and I decide not to choose at this point. A grayscale image contains only one channel, a colored image contains three. 
> to change from grayscale to color, just change the boolean parameter `color` in the beginning of this notebook.

The grayscale image is then simply converted to a vector by flattening the matrix, concatenating each row one after the others. 

The colored image is converted applying the same technique on the three channels, then concatenating the three resulting vectors into one, longer, vector.

####Resulting Dimensionality
From a training set colored image of dimensions $( h, w )$
- resize to $(64,64)$
- convert to vector:
    * if grayscale, convert to $(1, 4096)$
    * if color, convert to ($1, 12288)$

The resulting matrix representing the training set has a shape $(40,4096)$ or $(40,12288)$ depending if grayscale or color.

####Process the training set as a data matrix

In [0]:
"""
Build useable training set from (hyper-)parameters
"""

# column 0 = first image
# column 1 = second image
# ...
m_src = get_matrix_from_set(training_set, color, sq_size = sq_size, flatten = True)

logging.debug(" m_src: original matrix")
logging.debug(" m_src: shape = " + str(m_src.shape))

plot_matrix(m_src, color, my_color_map, h=4, w=10, transpose = False)

#### Process the test set as a data matrix

In [0]:
"""
Build useable test set from (hyper-)parameters
"""
m_test_src = get_matrix_from_set(test_set, color, sq_size = sq_size, flatten = True)


logging.debug(" m_test_src: original matrix")
logging.debug(" m_test_src: shape = " + str(m_test_src.shape))


plot_matrix(m_test_src, color, my_color_map,h = 4, w=10,transpose = False)

###PCA How-to, Step2: Centering the Data

Centering the data is essential in order to have, eventually, the eigenvectors sorted according to the eigenvalues properly meaning what we desire, aka the directions of max variances in the data.

The following code shows:
1. (left) a plot of the data (training) matrix 
2. (middle) the mean image
3. (right) a plot of the data where the mean image is substracted: the data are now centered


In [0]:
X = m_src.copy()
X_mean = np.mean(X, axis = 0)
Xc = X - X_mean

fig = plt.figure(figsize=(18, 5))

Xs = np.arange(0, X.shape[0])
Ys = np.arange(0, X.shape[1])
Xs, Ys = np.meshgrid(Xs, Ys)
  
ax1 = fig.add_subplot(131,projection='3d')
surf1 = ax1.plot_surface(Xs, Ys, X.T, cmap=plt.cm.jet, antialiased=True)
ax1.set_xlabel('image index')
ax1.set_ylabel('vector index')
ax1.set_zlabel('value')
ax1.set_title('Original data')
ax2 = fig.add_subplot(132)
ax2.imshow(my_reshape(X_mean, sq_size, color), cmap = my_color_map, interpolation='nearest') 
ax2.set_title("Mean image")
# ax2.imshow(X_mean.reshape(sq_size,sq_size), cmap=plt.cm.gray)

# ax2.plot(np.arange(0, m_src.shape[0]), m_src_means, 'bo')
# ax2.plot(np.arange(0, m_src.shape[1]), m_src_max, 'go')
# ax2.plot(np.arange(0, m_src.shape[1]), m_src_min, 'ro')

ax3 = fig.add_subplot(133,projection='3d')
surf3 = ax3.plot_surface(Xs, Ys, Xc.T, cmap=plt.cm.jet, antialiased=True)
ax3.set_xlabel('image index')
ax3.set_ylabel('vector index')
ax3.set_zlabel('value')
ax3.set_title("Centered data")

logging.info("Visualization of the images - centered:")
plot_matrix(Xc, color, my_color_map, h=4, w=10,transpose = False)
plt.show()

###PCA How-to, Step3: Decomposition


####*Canonical - EigenDecomposition*
####0. <u>Data </u>
Let's take our initial data matrix X, a $(n \times p)$ matrix of data where n is the number of images, and p is the number of variables. In our case, the number of variables is the number of pixels of one image (or three times this number, if color image)

First, we have to center the data, hence substracting the mean image. This is done in the previous step. In the following text, $X$ is assumed centered. 



####1. <u>Covariance Matrix </u>

Then, we can compute the covariance matrix, that indicates how a variable (= pixel intensity) varies with respect to other pixels. The Covariance matrix indicates how the variables evolve with respect to each others. 

$C = \frac{X^T \cdot X}{n-1}$ and has dimension $(p \times p)$. This is a pretty large matrix already.

####2. <u>Eigenvalues and EigenVectors </u>
Having computed the covariance matrix C, we can compute its eigenvectors and eigenvalues, indicating the main directions and their strength of how the data evolve. The eigendecomposition is expressed as:

$C = V * L * V^T$ where
- $L$ is a diagonal matrix of eigenvalues
- $V$ is the $(p \times p)$ matrix of eigenvectors


<u>*Mathematical Trick: Exploiting the dimensionality of the matrix*</u>

Computing the eigenvalues and eigenvectors of $C$ can be pretty cumbersome, as $C$ is a large matrix $(p \times p)$.

Recalling our algebra skills, given the dimension of $X$, only a limited amount of eigenvalues are non zero: there are only $(n-1)$ non zero eigenvalues. There is no need to compute the $p$ eigenvalues and related $(p \times p)$ eigenvectors matrix as (all) the information is contained in only $(n-1)$ eigenvalues.

As a result, to speed up the computation and take advantage of this property, instead of computing the eigenvalues and eigenvectors of $C = \frac{X^T \cdot X}{n-1}$ of size $(p \times p)$, let's rather compute the $n$ eigenvalues and corresponding eigenvectors of the matrix 
$D = \frac{X \cdot X^T}{n-1}$ 

of size $(n \times n)$, such that:

$D = W * L * W^T$
- the eigenvalues computed are the same of $C$
- the corresponding eigenvectors of C, in matrix $V$, are related such that $V = X^T \cdot W$

This way, it takes advantage of the dimensions of the problem.

####3. <u>Principal Components</u>

The principal components are defined as the eigenvectors $V$. The eigenvectors can be sorted according to the value of their associated eigenvalue, in decreasing order.

The eigenvector that has the largest eigenvalue associated is called "first principal component", the second largest is called "second principal component", and so on.

In the context of faces analysis, the eigenvectors are often called **eigenfaces**, as they can be reshaped as an image, and displayed (provided the appropriate number conversion so that it's in a range visible)

By projecting the original data $X$ on the new directions, the eigenfaces, we get *new coordinates* that yet *fully describe the original data*. 

Furthermore, the number of eigenvectors on which we project the data is reduced with respect to original problem dimensionality.   


####*Singular Value Decomposition*

The idea behind the SVD is essentially mathematical, and help in computing the eigenvectors and eigenvalues in a different way. From a matrix X, centered, (as in previous section), one can compute its decomposition $X = U \cdot S \cdot V^T$ 
where $U$ is a unitary matrix, $S$ is diagonal, containing what's called the singular values $s_i$. 
One can see that $V$, right singular vectors, are related to eigenvectors of the covariance matrix from previous section.

Indeed, computing this covariance matrix gives:

\begin{align}
Cov &= \frac{X^T \cdot X}{n-1} \\
    &= \frac{V \cdot S \cdot U^T \cdot U \cdot S \cdot V}{n-1} \\
    &= V \cdot \frac{S^2}{n-1} V^T
\end{align}

There is therefore a link between the singular values ($s_i$) and the eigenvalues ($\lambda_i$):
$$\lambda_i = \frac{s_i^2}{n-1}$$ and the right singular vectors are the eigenvectors $V$.

Note that in practice, most of the implementation of the PCA algorithm uses singular value decomposition, and starts by centering the data - such as the library function `sklearn.decomposition.PCA` that we will extensively use later on.

####Decomposition and Eigenfaces

The following lines of codes create an object of type `MyPCA`, and compute the decompositions following two methods: "svd" and "eigen_fast". 



In [0]:
nb_training_faces = sum(training_sets_size.values())

"""
Ensuring reset of object if cell is rerun
"""
my_pca = None
my_pca2 = None

"""
Getting the source matrix
"""
X = m_src.copy()

"""
Creating the MyPCA objects
-> solving according to SVD
-> solving according to EigenDecomposition (with Math Trick)
"""
my_pca = MyPCA("svd")
my_pca.fit(X)
print("All principal components, using SVD")
my_pca.show_principal_components(k=nb_training_faces)

my_pca2 = MyPCA("eigen_fast")
my_pca2.fit(X)
print("All principal components, using eigendecomposition")
my_pca2.show_principal_components(k=nb_training_faces)

# my_pca3 = MyPCA("eigen")
# my_pca3.fit(X)
# print("All principal components, using nominal eigendecomposition")
# my_pca3.show_principal_components(k=nb_training_faces)

The two plots above show the same information: the eigenfaces, but computed in two different ways: using SVD and using eigendecomposition. Several things are important to be noticed:
- the eigenfaces are very much alike. Actually, they are exactly the same (despite the last one, see next point) considering the well-known sign ambiguity related to decomposition, see [Standord course, sect. 5.3, Properties of eigenvectors](https://graphics.stanford.edu/courses/cs205a-13-fall/assets/notes/chapter5.pdf). Long story short, white and black may be reversed without any issue in the eigenfaces, on each image independantly. 
    * In the commonly used library implementation, there is often an `sign_flip`function implemented to ensure repeatability of the results of the decomposition. See [documentation](https://kite.com/python/docs/sklearn.utils.extmath.svd_flip)
- the last eigenface seems to differ... Definitely, this isn't an issue. 
    * recall that there are only (n - 1) non-zero eigenvalue. The eigenface associated to this eigenvalue is therefore multiplied by 0, and doesn't play a role. 

If interested, you may uncomment the last lines in order to create a PCA with parameter `method = "eigen"`, and see the result of the true eigendecomposition, without the mathematical trick. Or you can trust me that the result is the same (time to run ~50 seconds)

####Reconstructing on k components

In [0]:
"""
Selecting X as the input image we want to reconstruct
"""
X = m_src.copy()[9,:]

logging.debug("Shape of input X = " + str((X.shape)))

"""
Centering the data
"""
X_mean = np.mean(m_src.copy(), axis=0)
Xc = X - X_mean


"""
Computing X_reduced, projection of Xc on the principal components space
"""
X_reduced = my_pca.projectPC(Xc, k=nb_training_faces)
logging.debug("Shape of X_reduced = " + str(X_reduced.shape))

"""
Reconstructing progressively
Based on k first components only
"""
k=0
fig = plt.figure(figsize=(3,3))
img = my_reshape(X_mean, sq_size, color)
ax = fig.add_subplot(1, 1, 1, xticks=[], yticks=[]) 
ax.imshow(img, cmap = my_color_map, interpolation='nearest') 
plt.show()
logging.info("Principal components used = " + str(k) + ";\nReconstruction error = " + str(np.round(my_pca.compute_error(Xc+X_mean, X_mean),2)))

for k in [1,2,3,5,8,10,12,15,20,25,30,40]:
    X_hat_centered = my_pca.reconstruct(X_reduced, k, show=True)
    logging.info("\nAbove: \nPrincipal components used = " + str(k) + 
                 ";\nReconstruction error = " + 
                 str(np.round(my_pca.compute_error(Xc, X_hat_centered),2)) + 
                 "\n"+"___"*30)


# my_pca.show_data(X)
expl_var, cum_expl_var = my_pca.compute_explained_variance(show = True)


*Discussions*

On the above results and images, several things can be observed:
1. First the different reconstruction of one selected face. As expected, the reconstruction decreases as the number of components used (k) increases. This also match the intuition behind the explained variance.
2. Second, the ultimate error remaining is 0, indicating no information was lost when considering the 40 components. That confirms the mathematical theory.
3. The last two graphs show on the left, the eigenvalues, and on the right, the cumulative explained variance. 
    * the eigenvalues are sorted from the most important to the least important, confirming the right curve of the cumulative expl. var. However, the descent of the values is not fast (not exponential). Furthermore, there is no big drop off in the values.
    * this indicates that the choice of an **optimal number p** of components such that the dimensionality of the feature space is reduced but still informative is not obvious. 

####Choice of optimal $p$


$p$ is defined as the optimal number of components used in the reduced space so that:
1. the dimension is reduced (lower than n)
2. the reconstructed information is *close* to input data. It means the reconstruction is still informative.
Selecting only the first $p$ components has the effect of removing small variances. This can be important for some application, if there are little variance between different classes.


As said above:
- there is no clear drop-off in the eigenvalues,
- there is no exponential decrease if the eigenvalues.

It makes the choice of an optimal $p$ complicated. 

To choose, we will do:
1. compute the reconstruction loss (error):
    * for all training examples
    * for all possible choice of p
2. plot this RMSE, in absolute value, 
3. plot, in percentage, the ratio between the reconstruction loss using p-components and the RMSE between mean image and input image. This gives the notion of percentage of reconstruction error -- that can actually be related to the cumulative explained variance !
2. define a threshold of 95% of the information kept (5% of error tolerated)

That will lead to a sensible choice of $p$. This is however purely arbitrary.


In [0]:
X_train = m_src.copy()
X_train_mean = np.mean(X_train, axis = 0)
X_train_centered = X_train - X_train_mean

my_pca = None
my_pca = MyPCA("svd")
my_pca.fit(X_train)

n = X_train.shape[0]
"""
np array containing all the rmse computed

if n = 40, max number of components, then rmse has a size 40x40 (0 -> 39)
> a row matches the rmse of one image wrt the dimension reconstructed. Last column should be 0 (or close, ~e-14)
"""
rmse = np.empty((n,n+1))
rmse_pc = np.empty((n,n+1)) # rmse in percentage
rmse_base = np.empty((n,))
for i in range(n):
    rmse_base[i]=my_pca.compute_error(X_train[i], X_train_mean)


index_image = 0
for img_center_vector in X_train_centered:
    # logging.debug("projecting on " + str(n) + " principal components")
    X_centered_reduced = my_pca.projectPC(img_center_vector, n)
    for k in range(n+1):
        # from 1 to n, included
        if k == 0:
            rmse[index_image, k] = rmse_base[index_image]
            rmse_pc[index_image, k] = 100 * rmse[index_image,k] / rmse_base[index_image]
        else:
            # logging.debug("reconstructing using " + str(p) + " principal components")
            X_hat_centered = my_pca.reconstruct(X_centered_reduced, k)
            rmse[index_image,k] = my_pca.compute_error(img_center_vector, X_hat_centered)
            rmse_pc[index_image, k] = 100 * rmse[index_image,k] / rmse_base[index_image]
        
    index_image += 1

fig = plt.figure(figsize = (16,16))
ax1 = fig.add_subplot(2,2,1)

for idx in range(n):
    rmse_ = rmse[idx,:]
    ax1.plot([i for i in range(0,n+1)],rmse_, '-')
ax1.set_title("reconstruction errors (RMSE) for all images")
ax1.set_xlabel("reconstruction dimension(s) \'p\' ")
ax1.set_ylabel("RMSE")

rmse_mean = np.mean(rmse, axis = 0)
ax2 = fig.add_subplot(2,2,2)
ax2.plot([i for i in range(0, n+1)], rmse_mean, "ro-")
ax2.set_title("Mean of RMSE for all images")
ax2.set_xlabel("reconstruction dimension(s) \'p\' ")
ax2.set_ylabel("mean of RMSEs")

ax1.set_ylim((0,80))
ax2.set_ylim((0,80))
ax1.grid()
ax2.grid()

ax3 = fig.add_subplot(2,2,3)

for idx in range(n):
    rmse_ = rmse_pc[idx,:]
    ax3.plot([i for i in range(0,n+1)],rmse_, '-')
ax3.set_title("reconstruction errors (RMSE) for all images, in %")
ax3.set_xlabel("reconstruction dimension(s) \'p\' ")
ax3.set_ylabel("RMSE")

rmse_pc_mean = np.mean(rmse_pc, axis = 0)
ax4 = fig.add_subplot(2, 2,4)
ax4.plot([i for i in range(0, n+1)], rmse_pc_mean, "ro-")
ax4.set_title("Mean of RMSE for all images, in %")
ax4.set_xlabel("reconstruction dimension(s) \'p\' ")
ax4.set_ylabel("mean of RMSEs")

ax3.set_ylim((0,110))
ax4.set_ylim((0,110))
ax3.grid()
ax4.grid()

# print(rmse_pc_mean)

Following the explained process of defining $p$, the threshold is set at $p = 35$. This is definitely not an extraordinary result, but yet constitutes somehow a reduction, and based on a reasonable choice.

We can now visualize the training image reconstructed based on the first 35 components.

In [0]:
"""
Define optimal p
"""
p = 35

"""
Selecting X as the input image we want to reconstruct
"""
X = m_src.copy()
logging.debug("Shape of input X = " + str((X.shape)))

"""
Centering the data
"""
X_mean = np.mean(m_src.copy(), axis=0)
Xc = X - X_mean

"""
Creating data structure for outputs
"""
X_hat=np.empty(X.shape)


"""
Computing X_reduced, projection of Xc on the principal components space
"""
X_reduced = my_pca.projectPC(Xc, k=p)
logging.debug("Shape of X_reduced = " + str(X_reduced.shape))

"""
Reconstructing based on p first components only
"""
X_hat_centered = my_pca.reconstruct(X_reduced, p, show=False)
X_hat = X_hat_centered + X_mean
fig1 = plt.figure()
plot_matrix(X_hat, color, my_color_map, h=4, w=10, transpose=False)

"""
Visualize original input, for comparison purpose
"""
fig2=plt.figure()
fig2 = plot_matrix(X, color, my_color_map,h=4, w=10, transpose=False,return_figure=True)


for axis in ['top','bottom','left','right']:
    fig2.axes[1].spines[axis].set_linewidth(2)
    fig2.axes[1].spines[axis].set_color('white')



On the above two series of images, the top one is the reconstructed using 35 components, and the above one plots the original data.
- overall, the quality seems good enough to fully recognize all the faces pretty well, confirming that most of the variance is kept, and the remaining construction loss is small.
- several images, however, clearly show this loss (analysis in grayscale):
    * image[0,6] contains visibly some extra riddles on the bottom left of the face 
    * image[2,1] is appears difformed
    * image[3,6] shows reminiscence of hair on Bradley Cooper's forehead.
    * ...


Luckily, it still show some loss in the reconstruction, confirming the previous results established. Nonetheless, the quality is considered good enough to go on with the $p=35$ selected. 




####Plot on first two Principal Components
PCA is often used as dimensionality reduction technique to represent high dimensional data. Let's use that to show the faces on the first two principal components base.

*We do that first with the training images, reconstructed using p=35, for information. Later, we will apply the same thing on some test images*

In [0]:
my_pca = None

X = m_src.copy()
my_pca = MyPCA("svd")
my_pca.fit(X)

data = m_src.copy()
data_mean = np.mean(m_src.copy(), axis = 0)
data_centered = data - data_mean

# my_pca_svd.show_data(X)

# reduce the image to the principal components (k=2)
data_projected = my_pca.projectPC(data_centered, k=2)
logging.debug("shape of data_projected = " + str(data_projected.shape))


plt.figure() 
fig, ax = plt.subplots(1, 1, figsize=(14, 14), sharex=True, sharey=True)

eig1 = data_projected[:,0]
eig2 = data_projected[:,1]
ax.plot(eig1, eig2, 'bo')
ax.set_xlabel("First Component")
ax.set_ylabel("Second Component")

for (x_, y_), img_vector_ in zip(data_projected, X_hat):
    img = my_reshape(img_vector_, sq_size, color)
    ab = AnnotationBbox(OffsetImage(img, cmap = my_color_map), (x_, y_), frameon=False)
    ax.add_artist(ab)

ax.grid()
plt.show()

logging.info("\nIn order to better visualize the plot on top, here is the same view\nwere personA is in red, and personB is in green")

plt.figure() 
fig, ax = plt.subplots(1, 1, figsize=(8, 8), sharex=True, sharey=True)
ax.plot(eig1[0:20], eig2[0:20], 'ro')
ax.plot(eig1[20:40], eig2[20:40], 'go')
plt.show()


###Demo scikit learn

Of course, everything that has been done so far regarding PCA can be achieved using dedicated  - and optimized - libraries. For that purpose, we can use the `sklearn.decomposition` package that, among other things, implement the PCA using the *SVD* decomposition that we've looked at. 

A difference to note is the use, internally, of the `svd_flip(u, vt)`, a function that ensures the vectors to be deterministic, hence solving the sign ambiguity inherent to matrix decomposition, as already discussed above.

The following part first performs the same operation as we've implemented before to get familiarized.



In [0]:
"""
Let's start --again-- from the training set, processed as a matrix
"""
logging.info("shape of input matrix (n x p) = " + str(m_src.shape))
plot_matrix(m_src, color, my_color_map, h=4, w=10, transpose = False)

In [0]:
"""
Create pca sklearn object, and compute decomposition
"""
X = m_src.copy()
n_components = 40
logging.info("Shape input data: "+str(X.shape))
pca_ = sklearn_decomposition_PCA(n_components=n_components) 
pca_.fit(X)

"""
Get the eigenfaces
"""

eigenfaces = pca_.components_
logging.debug("eigenfaces shape = " + str(eigenfaces.shape))

"""
Visualize the eigenfaces, just as we did before
"""
plot_matrix(eigenfaces, color, my_color_map, h=4, w=10, transpose=False)


Without any surprise, we find again the same principal components as before.

Continuing with sklearn library, we can reconstruct the original data based on the first $p$ components. Hopefully, the results are the same as the ones obtained with the homemade PCA. 

In [0]:
from numpy.testing import assert_array_almost_equal
p=35
pca_ = sklearn_decomposition_PCA(n_components=p) 
pca_.fit(X)
X_reduced_sk = pca_.transform(X)
X_reconstructed_sk = pca_.inverse_transform(X_reduced_sk)

print(X_reconstructed_sk.shape)
plot_matrix(X_reconstructed_sk, color, my_color_map, h=4, w=10)
logging.info("Difference between Homemade PCA and Scikit-learn PCA: " + str(np.linalg.norm(X_hat - X_reconstructed_sk)))
logging.info(" ==> OK!")

### Projection of Test Faces reconstructed

Using the same kind of plot as before, we can reconstruct, **in the same eigenface base** the test set images using $p$ first components. 

faces (from the 4 persons) on the first two principal components


We need to apply the sames steps that were applied to the training set, to the test set. that is:


1.   centering the data: substracting the mean **of the training set**
2.   projecting the resulting centered data on the p first principal components computed by applying PCA on the training data. We won't fit the PCA to the test images, we *just* project the test data using the already-found principal components
3.   reconstructing the original data based on those p first principal components, and plot the result of the 40 images on the 2-first components space.



In [0]:
scatter_plot_3D = False
scatter_plot_3times = False

"""
Once again, start by locally copying the data structure
> data_test as new data
> data_train_mean for centering
"""

data_test = m_test_src.copy()
data_train_mean = np.mean(m_src.copy(), axis = 0)

data_test_centered = data_test - data_train_mean

data_test_projected = my_pca.projectPC(data_test_centered, k=p)
logging.debug("shape of data_projected = " + str(data_test_projected.shape))

"""
scatter plot in 3D - 3 first components
"""
if scatter_plot_3D:
    fig = plt.figure(figsize=(8,8))
    ax = fig.add_subplot(111, projection='3d')
    eig1 = data_test_projected[:,0]
    eig2 = data_test_projected[:,1]
    eig3 = data_test_projected[:,2]
    ax.scatter(eig1[0:10], eig2[0:10], eig3[0:10], 'b')
    ax.scatter(eig1[10:20], eig2[10:20], eig3[10:20], 'r')
    ax.scatter(eig1[20:30], eig2[20:30], eig3[20:30], 'g')
    ax.scatter(eig1[30:40], eig2[30:40], eig3[30:40], 'y')
    plt.show()

if scatter_plot_3times:
    fig = plt.figure(figsize=(24,8))

    eig1 = data_test_projected[:,0]
    eig2 = data_test_projected[:,1]
    eig3 = data_test_projected[:,2]


    ax1 = fig.add_subplot(1,3,1)
    eig1 = data_test_projected[:,0]
    eig2 = data_test_projected[:,1]
    eig3 = data_test_projected[:,2]
    ax1.plot(eig1[0:10], eig2[0:10], 'bo')
    ax1.plot(eig1[10:20], eig2[10:20], 'ro')
    ax1.plot(eig1[20:30], eig2[20:30], 'go')
    ax1.plot(eig1[30:40], eig2[30:40], 'yo')

    ax2 = fig.add_subplot(1,3,2)
    ax2.plot(eig1[0:10], eig3[0:10], 'bo')
    ax2.plot(eig1[10:20], eig3[10:20], 'ro')
    ax2.plot(eig1[20:30], eig3[20:30], 'go')
    ax2.plot(eig1[30:40], eig3[30:40], 'yo')

    ax3 = fig.add_subplot(1,3,3)
    ax3.plot(eig2[0:10], eig3[0:10], 'bo')
    ax3.plot(eig2[10:20], eig3[10:20], 'ro')
    ax3.plot(eig2[20:30], eig3[20:30], 'go')
    ax3.plot(eig2[30:40], eig3[30:40], 'yo')

    plt.show()

In [0]:
eig1 = data_test_projected[:,0]
eig2 = data_test_projected[:,1]

data_test_reconstructed = my_pca.reconstruct(data_test_projected, p, show=False)

plt.figure() 
fig, ax = plt.subplots(1, 1, figsize=(16, 16), sharex=True, sharey=True)
ax.grid()
ax.plot(eig1, eig2, 'bo')
for x_, y_, img_vector_ in zip(eig1, eig2, data_test_reconstructed):
    img = my_reshape(img_vector_ + data_train_mean, sq_size, color)
    ab = AnnotationBbox(OffsetImage(img, cmap = my_color_map), (x_, y_), frameon=False)
    ax.add_artist(ab)



plt.figure() 
fig, ax = plt.subplots(1, 1, figsize=(8, 8), sharex=True, sharey=True)
eig1 = data_test_projected[:,0]
eig2 = data_test_projected[:,1]
ax.plot(eig1[0:10], eig2[0:10], 'bo')
ax.plot(eig1[10:20], eig2[10:20], 'ro')
ax.plot(eig1[20:30], eig2[20:30], 'go')
ax.plot(eig1[30:40], eig2[30:40], 'yo')
ax.legend(["personA", "personB", "personC", "personD"])
plt.show()



The plots of the test reconstructions confirms the intuition behing the eigenfaces:
- Emma Stone, in blue, is mainly on the top; while Bradley Cooper, in red, in mainly on the bottom. They are well separated according to the 2nd eigenface, which seems related to the shape of the face, with a very dark part on the bottom right.
- Jane Leny, in green, a woman that resembles to Emma Stone for a human, is mainly on the top of the plot,
- Marc Blucas, in yellow, is a white man similar to Bradley Cooper. While half of the points are located in the lower left half, where most of Bradley cooper images also are, the rest of Marc Blucas images is in the middle of blue and green points.

Two two first eigenfaces, or principal components, already gives us some of the important information present in the data, even if their cumulative explained variance - fitted for the training set inputs - was actually not that high!

Although we will certainly not modify any hyper-parameters based on the test set images, it is still interesting to reproduce the metrics that we built for the training set and the choice of an optimal $p$. It is interesting to answer such questions as:
- what is the final relative reconstruction error ?
- How does it evolve with p ?
- Was $p$ a nice choice, regarding the test sets ?



In [0]:
"""
np array containing all the rmse computed

if n = 40, max number of components, then rmse has a size 40x40 (0 -> 39)
> a row matches the rmse of one image wrt the dimension reconstructed. 
Last column should NOT be 0 as we compute construction of test images (= with loss)
"""
n = m_test_src.shape[0]

rmse_test = np.empty((n,n+1))
rmse_test_pc = np.empty((n,n+1)) # rmse in percentage
rmse_test_base = np.empty((n,))
for i in range(n):
    rmse_test_base[i]=my_pca.compute_error(data_test[i], X_train_mean)


index_image = 0
for img_center_vector in data_test_centered:
    X_test_centered_reduced = my_pca.projectPC(img_center_vector, n)
    for k in range(n+1):
        # from 1 to n, included
        if k == 0:
            rmse_test[index_image, k] = rmse_test_base[index_image]
            rmse_test_pc[index_image, k] = 100 * rmse_test[index_image,k] / rmse_test_base[index_image]
        else:
            # logging.debug("reconstructing using " + str(p) + " principal components")
            X_test_hat_centered = my_pca.reconstruct(X_test_centered_reduced, k)
            rmse_test[index_image,k] = my_pca.compute_error(img_center_vector, X_test_hat_centered)
            rmse_test_pc[index_image, k] = 100 * rmse_test[index_image,k] / rmse_test_base[index_image]
        
    index_image += 1

"""
Visualization !
"""

fig = plt.figure(figsize = (16,16))
ax1 = fig.add_subplot(2,2,1)

for idx in range(n):
    rmse_ = rmse_test[idx,:]
    ax1.plot([i for i in range(0,n+1)],rmse_, '-')
ax1.set_title("reconstruction errors (RMSE) for all test images")
ax1.set_xlabel("reconstruction dimension(s) \'p\' ")
ax1.set_ylabel("RMSE")

rmse_test_mean = np.mean(rmse_test, axis = 0)
ax2 = fig.add_subplot(2,2,2)
ax2.plot([i for i in range(0, n+1)], rmse_test_mean, "ro-")
ax2.set_title("Mean of RMSE for all test images")
ax2.set_xlabel("reconstruction dimension(s) \'p\' ")
ax2.set_ylabel("mean of RMSEs")

ax1.set_ylim((0,80))
ax2.set_ylim((0,80))
ax1.grid()
ax2.grid()

ax3 = fig.add_subplot(2,2,3)

for idx in range(n):
    rmse_ = rmse_test_pc[idx,:]
    ax3.plot([i for i in range(0,n+1)],rmse_, '-')
ax3.set_title("reconstruction errors (RMSE) for all test images, in %")
ax3.set_xlabel("reconstruction dimension(s) \'p\' ")
ax3.set_ylabel("RMSE")

rmse_test_pc_mean = np.mean(rmse_test_pc, axis = 0)
ax4 = fig.add_subplot(2, 2,4)
ax4.plot([i for i in range(0, n+1)], rmse_test_pc_mean, "ro-")
ax4.set_title("Mean of RMSE for all test images, in %")
ax4.set_xlabel("reconstruction dimension(s) \'p\' ")
ax4.set_ylabel("mean of RMSEs")

ax3.set_ylim((0,110))
ax4.set_ylim((0,110))
ax3.grid()
ax4.grid()




*Observations*
- the reconstruction loss, computed in the same fashion as for the training set images, decreases also as $p$ increases
- the slope seems to become near 0 as $p$ is ~35. It comforts us with the choice of $p=35$
- using all the components, the remaining error in the reconstruction is still 60% of the error of the base error. there is "no way" to do better.
> as a reminder, the "base" error is the RMSE between the input image, and the mean image of the training set. 

# Exploit Feature Representations
- use of feature representations that we have constructed before
- evaluate each feature representation for classification and identification
- discuss each method + why it is appropriate

## Classification - howto

- pre-process data
    1. Resizing to appropriate size => selected parameter
    2. color or grayscale => selected parameter
    
- compute feature reprensentation 
    1. HOG
    2. PCA
- train the corresponding classifier using training sets
    1. training using HOG feature
    2. training using PCA feature
- apply the classifier on unseen images (test)
    * apply same preprocessing as for training (exact same)
    * train
    * compute metrics


### Preprocess the data

- get raw training data: faces cropped data for personA and personB
- resize the raw images to a common image size


In [0]:
logging.info(min(get_min_size(training_set), get_min_size(test_set)))

The smallest face crop that we have in training set and test set is (70,70), so we can without issue rescale all our face crops to (64,64) as part of the data pre-processing.

In [0]:
X_train = get_matrix_from_set(training_set, color, sq_size = sq_size, flatten = False)
X_test = get_matrix_from_set(test_set, color, sq_size = sq_size, flatten = False)
y_train = np.zeros((40,))
y_train[20:40] = 1
 
"""
For now, set up "0" for personC; "1" for personD !
"""
y_test = np.zeros((40,))
y_test[10:20] = 1
y_test[30:40] = 1


In [0]:
logging.info("Training set (horizontal axis):")
logging.info("Test set PersonA [0 -> 19]:")
plot_matrix(X_train[0:20,:], color, my_color_map, h=1, w=20, transpose = False)
logging.info("Test set PersonA [20 -> 39]:")
plot_matrix(X_train[20:40,:], color, my_color_map, h=1, w=20, transpose = False)

logging.info("Test set (vertical axis):")
logging.info("Test set PersonA [0 -> 9]:")
plot_matrix(X_test[0:10, :], color, my_color_map, h=1, w=10, transpose = False)
logging.info("Test set PersonB [10 -> 19]:")
plot_matrix(X_test[10:20, :], color, my_color_map, h=1, w=10, transpose = False)
logging.info("Test set PersonC [20 -> 29]:")
plot_matrix(X_test[20:30, :], color, my_color_map, h=1, w=10, transpose = False)
logging.info("Test set PersonD [30 -> 39]:")
plot_matrix(X_test[30:40, :], color, my_color_map, h=1, w=10, transpose = False)

## HOG Classification


### Construction of Transformers
followed method of https://kapernikov.com/tutorial-image-classification-with-scikit-learn/

In [0]:
from sklearn.base import BaseEstimator, TransformerMixin

class HogTransformer(BaseEstimator, TransformerMixin):
    """
    Expects an array of 2D arrays (1 channel images)
    Calculates hog features for each image
    """
    def __init__(self, y=None, 
                 orientations = 9, 
                 pixels_per_cell = (8,8), 
                 cells_per_block = (2,2),
                 block_norm = "L2-Hys", 
                 transform_sqrt = False,
                 multichannel = False):
        self.y = y
        self.orientations = orientations
        self.pixels_per_cell = pixels_per_cell
        self.cells_per_block = cells_per_block
        self.block_norm = block_norm
        self.transform_sqrt = transform_sqrt
        self.multichannel = multichannel # default is grayscale
    
    def fit(self, X, y=None):
        logging.debug("[HOGTransformer.fit] X.Shape " + str(X.shape))
        return self
    
    def transform(self, X, y=None):
        logging.debug("[HOGTransformer.transform] X.Shape " + str(X.shape))

        def local_hog(X):
            if self.multichannel:
                X_ = X.copy().T
                logging.debug("TO CHECK IF TRANSPOSE STILL NEEDED ?")
                logging.debug("[HOGTransformer.transform] (1) X.Shape " + str(X.shape))
                logging.debug("[HOGTransformer.transform] (2) X_.Shape " + str(X_.shape))
            else:
                X_ = X.copy()
            # logging.debug("[HOGTransformer.transform.local_HOG]" )
            # cv2_imshow(X_)
            return skimage_feature_hog(X_,
                                       orientations = self.orientations, 
                                       pixels_per_cell = self.pixels_per_cell,
                                       cells_per_block = self.cells_per_block,
                                       block_norm = self.block_norm,
                                       visualize = False, 
                                       transform_sqrt = self.transform_sqrt, 
                                       feature_vector = True, 
                                       multichannel = self.multichannel)

        try: 
            # tmp = [str(image.shape) for image in X]
            # logging.debug( str(tmp) )
            return np.array([local_hog(image) for image in X])
        except ValueError as ve:
            logging.error(str(ve))
        except NameError as ne:
            logging.error(str(ne))


HOG feature


In [0]:
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import cross_val_predict
from sklearn.preprocessing import StandardScaler

# resizify = ResizeTransformer(sq_size=sq_size)
# grayify = RGB2GrayTransformer(color=color)

# scalify = StandardScaler()
hogify = HogTransformer(    orientations = 9, 
                            pixels_per_cell = (16,16), 
                            cells_per_block = (2,2),
                            block_norm = "L2-Hys", 
                            transform_sqrt = False,
                            multichannel = color)



# X_train_gray = grayify.fit_transform(X_train)
X_train_hog = hogify.fit_transform(X_train)
# X_train_prepared = scalify.fit_transform(X_train_hog)
X_train_prepared = X_train_hog

In [0]:
print(X_train_prepared.shape)

In [0]:
sgd_clf = SGDClassifier(random_state=42, max_iter=1000, tol=1e-6, verbose = 1)
sgd_clf.fit(X_train_prepared, y_train)

In [0]:

X_test_ab = X_test[0:20,:]
y_test_ab = y_test[0:20]
print("X_test shape -> " + str(X_test.shape))
print("X_test_ab shape -> " + str(X_test_ab.shape))
print("ytest [10]      -> " + str(sum(y_test_ab)))
X_test_hog = hogify.transform(X_test_ab)
X_test_prepared = X_test_hog

y_pred = sgd_clf.predict(X_test_prepared)
logging.info("Percentage correct: " + str( 100* np.sum(y_pred == y_test_ab)/len(y_test_ab) ))

In [0]:
from sklearn.metrics import confusion_matrix

label_names = ["Emma Stone", "Bradly Cooper"]

cmx = confusion_matrix(y_test_ab, y_pred)
cmx


In [0]:
from mpl_toolkits.axes_grid1 import make_axes_locatable
 
def plot_confusion_matrix(cmx, vmax1=None, vmax2=None, vmax3=None):
    cmx_norm = 100*cmx / cmx.sum(axis=1, keepdims=True)
    cmx_zero_diag = cmx_norm.copy()
 
    np.fill_diagonal(cmx_zero_diag, 0)
 
    fig, ax = plt.subplots(ncols=3)
    fig.set_size_inches(12, 3)
    [a.set_xticks(range(6)) for a in ax]
    [a.set_yticks(range(6)) for a in ax]
 
    im1 = ax[0].imshow(cmx, vmax=vmax1)
    ax[0].set_title('as is')
    im2 = ax[1].imshow(cmx_norm, vmax=vmax2)
    ax[1].set_title('%')
    im3 = ax[2].imshow(cmx_zero_diag, vmax=vmax3)
    ax[2].set_title('% and 0 diagonal')
 
    dividers = [make_axes_locatable(a) for a in ax]
    cax1, cax2, cax3 = [divider.append_axes("right", size="5%", pad=0.1)
                        for divider in dividers]
 
    fig.colorbar(im1, cax=cax1)
    fig.colorbar(im2, cax=cax2)
    fig.colorbar(im3, cax=cax3)
    fig.tight_layout()
 
plot_confusion_matrix(cmx)

Now, we have a quite nice score, but several parameters. U
Use of pipeline in order to build a *gridsearch* ...
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXx

### HOG/Classification - OPTIMIZATION


In [0]:
from sklearn.pipeline import Pipeline
from sklearn import svm

"""
Definition of a pipeline
"""
HOG_pipeline = Pipeline([
                         ('hogify', HogTransformer(
                             orientations = 9,
                             pixels_per_cell = (4,4),
                             cells_per_block = (2,2),
                             block_norm='L1',
                             transform_sqrt=False, 
                             multichannel = color)
                         ),
                         ('classify', SGDClassifier(
                             random_state = 42, 
                             max_iter = 1000, 
                             tol=1e-3)
                         )

])

clf = HOG_pipeline.fit(X_train, y_train)
# logging.info("Percentage correct: " + str( 100* np.sum(y_pred == y_test_ab)/len(y_test_ab) ) + " %")
y_pred = clf.predict(X_test) 
logging.info("Percentage correct: " + str( 100*np.sum(y_pred == y_test)/len(y_test)) + " %")
print(y_pred)

In [0]:
from sklearn.model_selection import GridSearchCV
 
param_grid = [
    {
        'hogify__orientations': [9],
        'hogify__cells_per_block': [(2, 2),(3, 3)],
        'hogify__pixels_per_cell': [(4,4),(8, 8),(12,12), (16, 16)],
        'hogify__block_norm': ['L1', 'L2', 'L2-Hys'],
        'hogify__transform_sqrt': [False, True],
        'hogify__multichannel':[color],
        'classify': [
            SGDClassifier(random_state=42, max_iter=1000, tol=1e-3),
            svm.SVC(kernel='linear'),
            svm.SVC(kernel='poly')]}
]

grid_search = GridSearchCV(HOG_pipeline,
                           param_grid, 
                           cv = 4, 
                           n_jobs = -1,
                           scoring = "accuracy",
                           verbose = 1, 
                           return_train_score = True)

grid_res = grid_search.fit(X_train, y_train)

In [0]:
grid_res.best_estimator_

In [0]:
grid_res.best_score_

In [0]:
grid_res.best_params_

In [0]:
best_prediction = grid_res.predict(X_test)
print('Percentage correct: ', 100*np.sum(best_prediction == y_test)/len(y_test))

In [0]:
idx = np.where( np.array(y_test - best_prediction) != 0)
print(idx)
print(y_test)

## PCA Classification

In [0]:
X_train = get_matrix_from_set(training_set, color, sq_size, flatten=True)
X_test= get_matrix_from_set(test_set, color, sq_size, flatten = True)

logging.debug("X_train_PCA Shape = " + str(X_train.shape))
logging.debug("X_test_PCA Shape = " + str(X_test.shape))

y_train = y_train.copy()
y_test = y_test.copy()

logging.debug("y_train_PCA Shape = " + str(y_train.shape))
logging.debug("y_test_PCA Shape = " + str(y_test.shape))

In [0]:
pcaify = sklearn_decomposition_PCA(n_components = 25)
X_train_PCA = pcaify.fit_transform(X_train)
X_train_prepared = X_train_PCA

print(X_train_prepared.shape)

In [0]:
sgd_clf = SGDClassifier(random_state=42, max_iter = 1000, tol=1e-4, verbose = 1)
sgd_clf.fit(X_train_prepared, y_train)

In [0]:
X_test_PCA = pcaify.transform(X_test)
X_test_prepared = X_test_PCA
y_pred = sgd_clf.predict(X_test_prepared)

logging.info("Percentage correct: " + str(100 * np.sum(y_pred == y_test)/len(y_test)))
cmx = confusion_matrix(y_test, y_pred)
plot_confusion_matrix(cmx)

In [0]:
"""
Definition of a PCA pipeline
"""
PCA_pipeline = Pipeline([
                         ('pcaify', sklearn_decomposition_PCA(
                             n_components = 25)
                         ),
                         ('classify', SGDClassifier(
                             random_state = 42, 
                             max_iter = 1000, 
                             tol=1e-3)
                         )

])

clf = PCA_pipeline.fit(X_train, y_train)
# logging.info("Percentage correct: " + str( 100* np.sum(y_pred == y_test_ab)/len(y_test_ab) ) + " %")
y_pred = clf.predict(X_test) 
logging.info("Percentage correct: " + str( 100*np.sum(y_pred == y_test)/len(y_test)) + " %")
print(y_pred)

In [0]:
param_grid = [
    {
        'pcaify__n_components': range(1,41,2),
        'classify': [
            SGDClassifier(random_state=42, max_iter=1000, tol=1e-3),
            svm.SVC(kernel='linear'),
            svm.SVC(kernel='poly')]}
]

grid_search = GridSearchCV(PCA_pipeline,
                           param_grid, 
                           cv = 3, 
                           n_jobs = -1,
                           scoring = "accuracy",
                           verbose = 1, 
                           return_train_score = True)

grid_res = grid_search.fit(X_train, y_train)

In [0]:
logging.info("Best Score -> " + str(grid_res.best_score_))

In [0]:
logging.info("Best Parameters found:")
logging.info(grid_res.best_params_)


In [0]:
best_prediction = grid_res.predict(X_test)
logging.info("Percentage correct: " + str(100*np.sum(best_prediction == y_test)/len(y_test)))

In [0]:
idx = np.where( np.array(y_test-best_prediction) != 0)
logging.debug(idx)
logging.debug(best_prediction)
logging.debug(y_test)
# cv2_imshow(np.reshape(X_test[22,:], (64,64,3)))
# cv2_imshow(np.reshape(X_test[23,:], (64,64,3)))
# cv2_imshow(np.reshape(X_test[25,:], (64,64,3)))
# cv2_imshow(np.reshape(X_test[26,:], (64,64,3)))
# cv2_imshow(np.reshape(X_test[27,:], (64,64,3)))
# cv2_imshow(np.reshape(X_test[35,:], (64,64,3)))
# cv2_imshow(np.reshape(X_test[39,:], (64,64,3)))

## Identification
In an identification setup the goal is to **compute similarity scores** between pairs of data examples and use them to identify new images. 
In this exercise you will:
- **compute the feature representation of every image** and 
- **create a data space using your training data**, 
- **give each of the data points a label based on the person they contain**.

Next try to **identify the test images using k-NN**. 

* What value did you use for k? 
* Was the identification of person A and B successful for all descriptors? 
* How did images of person C and D get labeled?


===
My understanding:
> * stepA: compute for, all the training set images, the handcrafted feature representation selected (for instance, HOG)
> * stepB: take one particular test image, compute its HOG
> * stepC: compute similarity score between HOG_test_image and all other training set images HOG representations computed in stepA
> * stepD: label according to k-NN
> * stepE: repeat for all other test images (including for personC and personD)

In [0]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import cosine_distances
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.metrics.pairwise import manhattan_distances



```
>>> from numpy.linalg import norm
>>> norm(X, axis=1, ord=1)  # L-1 norm
>>> norm(X, axis=1, ord=2)  # L-2 norm
>>> norm(X, axis=1, ord=np.inf)  # L-∞ norm
```




In [0]:
def plot_similarity_matrix(similarity_matrix, show_numbers = False, vmax1 = None, vmax2 = None, norm_only=False, width=16, height = 8, fontsize = 10):
    similarity_matrix_norm = 100*similarity_matrix / np.linalg.norm(similarity_matrix, axis = 1, ord=1, keepdims=True)

    if norm_only:
        raise NotImplementedError    
    else:
        fig, ax = plt.subplots(ncols=2)
        fig.set_size_inches(width, height)
    
        im1 = ax[0].imshow(similarity_matrix, vmax=vmax1, cmap="jet")
        ax[0].set_title('as is')
        im2 = ax[1].imshow(similarity_matrix_norm, vmax=vmax2, cmap="jet")
        ax[1].set_title('%')
        dividers = [make_axes_locatable(a) for a in ax]
        cax1, cax2 = [divider.append_axes("right", size="5%", pad=0.1) for divider in dividers]
    
        fig.colorbar(im1, cax=cax1)
        fig.colorbar(im2, cax=cax2)

        if show_numbers:
            for i in range(similarity_matrix.shape[0]):
                for j in range(similarity_matrix.shape[1]):
                    ax[0].text(j, i, str(similarity_matrix[i,j]), fontsize=fontsize,va='center', ha='center')
                    ax[1].text(j, i, str(round(similarity_matrix_norm[i,j],0)).rstrip('.0'),fontsize=fontsize,va='center', ha='center')
    fig.tight_layout()
    plt.show()

#### Illustration of distance measures


In order to better understand the vizualisation used, a tool example is given hereunder. The input is a matrix handmade. Later, it will be the matrix of shape (#test_image, #train_image) containing the similarity (or reversely, distance) measures pairwise.

Regarding the input matrix:
- on each line, the ratio between number is kept the same
- the five rows contain three different scale of the numbers:    

```
[[ 100,  20,  30,  40,  50], 
 [   2,  10,   3,   4,   5], 
 [  20,  30, 100,  40,  50],
 [   2,   3,   4,  10,   5], 
 [ 200, 300, 400, 500,1000]]
```


The numbers written indicate the value of the cell.

**Left side**: The "As is" matrix colors the cell as the numbers are set. The colorscale is therefore really large, and it can be useful to observe the disparity of measures across all tests. 

If the represented matrix is a distance matrix, for instance, one can observe that the 5th test image is *very far* from all the training images, as the 4th row as larger numbers than any other row. 
The drawback is that if some numbers are much higher than others, we lose in granularity to represent the differences between those numbers. For instance, a distance of "2" and a distance of "20" are very much alike.

**Right side**: The figure shows the same input matrix but normalized by row. That means that the sum of all numbers in a row is equal to 100 %. 
It is helpful to analyze *locally* the distance/similarity values for 1 specific test image with respect to all training images. 

In [0]:
test_dist_mtx = np.array([[100,20,30,40,50], [2,10,3,4,5], [20,30,100,40,50],[2,3,4,10,5], [200,300,400,500,1000]])
plot_similarity_matrix(test_dist_mtx, show_numbers=True, width=8, height=4, fontsize=14)

In the next sections of this tutorial, these vizualisations will be used to detail the similarity/distance measures between our different images. 
The vertical axis correspond to test images, with the following mapping
- 0 -> 9: images of PersonA, Emma Stone
- 10 -> 19: images of PersonB, Bradley Cooper
- 20 -> 29: images of PersonC, Jane Levy
- 30 -> 39: images of PersonD, Marc Blucas

The horizontal axis corresponds to the training images, with the following mapping:
- 0 -> 19: images of PersonA (training set)
- 20 -> 39: images of PersonB (training set)

If the feature descritions are appropriate, the features distance measurements should lead to
- very small distance (= large similarity measure) between training and test images of PersonA; and similarly for personB)
- very high distance (= small similarity measure) between training and test images of PersonA; and similarly for personB)

==> Persons from the same class would share feature descriptions, and not share other class feature description

Intuitively:
- feature description of personC should be closer (less distant, more similar) to personA than personB;
- feature descriptions of personD should be closer to personB than personA.

### based on HOG feature decriptors





#### Pre-process data

In [0]:
X_train = get_matrix_from_set(training_set, color, sq_size = sq_size, flatten = False)
X_test = get_matrix_from_set(test_set, color, sq_size = sq_size, flatten = False)

y_train = np.zeros((40,))
y_train[20:40] = 1
 
"""
For now, set up "0" for personC; "1" for personD !
"""
y_test = np.zeros((40,))
y_test[10:20] = 1
y_test[30:40] = 1


In [0]:
logging.info("Training set (horizontal axis):")
logging.info("Test set PersonA [0 -> 19]:")
plot_matrix(X_train[0:20,:], color, my_color_map, h=1, w=20, transpose = False)
logging.info("Test set PersonA [20 -> 39]:")
plot_matrix(X_train[20:40,:], color, my_color_map, h=1, w=20, transpose = False)

logging.info("Test set (vertical axis):")
logging.info("Test set PersonA [0 -> 9]:")
plot_matrix(X_test[0:10, :], color, my_color_map, h=1, w=10, transpose = False)
logging.info("Test set PersonB [10 -> 19]:")
plot_matrix(X_test[10:20, :], color, my_color_map, h=1, w=10, transpose = False)
logging.info("Test set PersonC [20 -> 29]:")
plot_matrix(X_test[20:30, :], color, my_color_map, h=1, w=10, transpose = False)
logging.info("Test set PersonD [30 -> 39]:")
plot_matrix(X_test[30:40, :], color, my_color_map, h=1, w=10, transpose = False)

Creation of Transformer

In [0]:
hogify = None
hogify = HogTransformer(    orientations = 9, 
                            pixels_per_cell = (16,16), 
                            cells_per_block = (2,2),
                            block_norm = "L2-Hys", 
                            transform_sqrt = False,
                            multichannel = color)


Transformation of Inputs
 - **X_train**: application of the fit then transform method from the transformer
 - **X_test**: application of the transform method from the transformer



In [0]:
X_train_hog = hogify.fit_transform(X_train)
X_test_hog = hogify.transform(X_test)

logging.debug("X_train_hog Shape: " + str(X_train_hog.shape))
logging.debug("X_test_hog Shape: " + str(X_test_hog.shape))


#### compute similarities pairwise
using cosine similarities
=> explain why

In [0]:
# hog_similarities = cosine_similarity(X_test_hog, X_train_hog)
hog_distances_cos = cosine_distances(X_test_hog, X_train_hog)
hog_distances_eucl = euclidean_distances(X_test_hog, X_train_hog)
# plot_similarity_matrix(hog_similarities)
logging.debug("DISTANCE BASED ON COSINE")
plot_similarity_matrix((hog_distances_cos))

logging.debug("DISTANCE BASED ON COSINE - LOG10")
plot_similarity_matrix(np.log10(hog_distances_cos))

logging.debug("DISTANCE BASED ON EUCL")
plot_similarity_matrix((hog_distances_eucl))

logging.debug("DISTANCE BASED ON EUCL - LOG10")
plot_similarity_matrix(np.log10(hog_distances_eucl))



kNN


In [0]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics

k=1
knn = KNeighborsClassifier(n_neighbors = k, metric='euclidean')
knn.fit(X_train_hog, y_train)
y_pred_hog = knn.predict(X_test_hog)
logging.info("Percentage correct: " + str(100*np.sum( y_test == y_pred_hog)/len(y_test)))
logging.info("Percentage correct [SKlearn]: " + str(metrics.accuracy_score(y_test, y_pred_hog)))
logging.debug("y_pred => " + str(y_pred_hog))
print(knn.kneighbors(X_test_hog[0,:].reshape(1, -1), n_neighbors=40))


idées : 
> selectionner les images de tests et montrer leurs nearest neighbours; 
    - quand ça va bien
    - quand ça va pas bien



### based on PCA feature descriptors


In [0]:
X_train = get_matrix_from_set(training_set, color, sq_size = sq_size, flatten = True)
X_test = get_matrix_from_set(test_set, color, sq_size = sq_size, flatten = True)
y_train = np.zeros((40,))
y_train[20:40] = 1
 
"""
For now, set up "0" for personC; "1" for personD !
"""
y_test = np.zeros((40,))
y_test[10:20] = 1
y_test[30:40] = 1

In [0]:
logging.info("Training set (horizontal axis):")
logging.info("Test set PersonA [0 -> 19]:")
plot_matrix(X_train[0:20,:], color, my_color_map, h=1, w=20, transpose = False)
logging.info("Test set PersonA [20 -> 39]:")
plot_matrix(X_train[20:40,:], color, my_color_map, h=1, w=20, transpose = False)

logging.info("Test set (vertical axis):")
logging.info("Test set PersonA [0 -> 9]:")
plot_matrix(X_test[0:10, :], color, my_color_map, h=1, w=10, transpose = False)
logging.info("Test set PersonB [10 -> 19]:")
plot_matrix(X_test[10:20, :], color, my_color_map, h=1, w=10, transpose = False)
logging.info("Test set PersonC [20 -> 29]:")
plot_matrix(X_test[20:30, :], color, my_color_map, h=1, w=10, transpose = False)
logging.info("Test set PersonD [30 -> 39]:")
plot_matrix(X_test[30:40, :], color, my_color_map, h=1, w=10, transpose = False)


In [0]:
pcaify = None
pcaify = sklearn_decomposition_PCA(n_components = 35)

X_train_PCA = pcaify.fit_transform(X_train)
X_test_PCA = pcaify.transform(X_test)


# pca_similarities = cosine_similarity(X_test_PCA, X_train_PCA)
pca_distances_cos = cosine_distances(X_test_PCA, X_train_PCA)
pca_distances_eucl = euclidean_distances(X_test_PCA, X_train_PCA)
# plot_similarity_matrix(pca_similarities)
# plot_similarity_matrix(np.log(pca_distances_cos))
plot_similarity_matrix((pca_distances_eucl))


In [0]:
k=1
knn = KNeighborsClassifier(n_neighbors = k, metric='euclidean')
knn.fit(X_train_PCA, y_train)
y_pred_PCA = knn.predict(X_test_PCA)
logging.info("Percentage correct: " + str(100*np.sum( y_test== y_pred_PCA )/len(y_test)))
logging.info("Percentage correct [SKlearn]: " + str(metrics.accuracy_score(y_test, y_pred_PCA )))
logging.debug("Y Pred => " + str(y_pred_PCA))
print(knn.kneighbors(X_test_PCA [0,:].reshape(1, -1), n_neighbors=40))

## Observations and Comments
- PCA distance measures seem more fuzzy than HOG feature descriptors
